## Passos a Seguir
- Importar os dados de Treino e de Teste
- Retirar as criticas e as classificações, tanto dos dados de Treino como de Teste.
    - X1 - Criticas Treino
    - y1Feel, y1Look, y1Smell, y1Taste, y1Overall - Treino
    <br>
    <br>
    - X2 - Criticas Teste
    - y2Feel, y2Look, y2Smell, y2Taste, y2Overall - Teste
- Existem dois tipos de classificação por fazer:
    - Binária: Uma classe é para o Overall entre 1 e 2, a outra é para o Overall entre 9 e 10.
    - Multi-casse: Esta já envolve os valores de todas as classificações individuais (1 a 9). Para o Overall, Smell e Taste.
- Devem ser criados dois dicionários, um para a classificação binária e outro para a multi-classe.

In [141]:
import pickle
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from sklearn.svm import SVC
import time

In [142]:
D_train = pickle.load(open('rateBeer75Ktrain.p', 'rb'))
D_test = pickle.load(open('rateBeer25Ktest.p', 'rb'))

In [9]:
print(D_train['000000'].keys())

dict_keys(['name', 'style', 'who', 'feel', 'look', 'smell', 'taste', 'overall', 'review'])


In [143]:
#Arranjar os X's do treino.
X1 = np.array([valor["review"] for valor in D_train.values()])

#Arranjar os y's do treino.
y1Feel = np.array([int(valor["feel"][0]) for valor in D_train.values()])
y1Look = np.array([int(valor["look"][0]) for valor in D_train.values()])
y1Smell = np.array([int(valor["smell"][0]) for valor in D_train.values()])
y1Taste = np.array([int(valor["taste"][0]) for valor in D_train.values()])
y1Overall = np.array([int(valor["overall"][0]) for valor in D_train.values()])

#Arranjar os X's de teste.
X2 = np.array([valor["review"] for valor in D_test.values()])

#Arranjar os y's do teste.
y2Feel = np.array([int(valor["feel"][0]) for valor in D_test.values()])
y2Look = np.array([int(valor["look"][0]) for valor in D_test.values()])
y2Smell = np.array([int(valor["smell"][0]) for valor in D_test.values()])
y2Taste = np.array([int(valor["taste"][0]) for valor in D_test.values()])
y2Overall = np.array([int(valor["overall"][0]) for valor in D_test.values()])

In [11]:
#Observar os Dados de Treino
print(X1.shape)
print(y1Feel.shape)
print(y1Look.shape)
print(y1Smell.shape)
print(y1Taste.shape)
print(y1Overall.shape)

print()

print(X1)
print(y1Feel)
print(y1Look)
print(y1Smell)
print(y1Taste)
print(y1Overall)

(75000,)
(75000,)
(75000,)
(75000,)
(75000,)
(75000,)

['22oz.  Pours dark brown with a red rim and off-white head.  Nose is mostly sweet malt with fairly noticeable alcohol depending on the temperature.  Flavor is sweet malt with raisin and banana yeast with some slight spice on the finish.  Very true to the style, but less than inspiring.'
 'Compliments of a trade with goldtwins, thanks!\tPoured dark with very little head.  Coffee bean and chocolate dominate from the start, slight sweetness in the middle and finishes with a bit of bitterness from the coffee.  A nice hefty porter for the winter months.'
 'Pours a dark brown with a 1/2 inch head.  Aroma is roasty with some coffee and chocolate, but smells very lager-ish.  Perhaps a little too much hop.  Flavor is roasty, but the hop adds too much to that.  Almost tastes slightly burnt.  Not much else going on in the flavor department.  Not a bad beer in the least, but not terribly far above average.'
 ...
 'Bottle at Kernel Brewery tas

In [13]:
#Observar os Dados de Teste
print(X2.shape)
print(y2Feel.shape)
print(y2Look.shape)
print(y2Smell.shape)
print(y2Taste.shape)
print(y2Overall.shape)

print()

print(X2)
print(y2Feel)
print(y2Look)
print(y2Smell)
print(y2Taste)
print(y2Overall)

(25000,)
(25000,)
(25000,)
(25000,)
(25000,)
(25000,)

['Tap, Toronado during CBC week.  Good oyster flavor in this beer, but the roast and hops came through too harshly for my tastes.  Earthy and a bit mineraly.  I wish that I had nabbed a bottle because I imagine that this one had a good chance of settling down and letting those salty, earthy notes combine with more of the dark chocolate lurking beneath the roast.'
 'Courtesy of Degarth. Pours dark mahogany with tan head. The aroma is big with earthiness mixed with caramel covered apples, prunes, the salty seaweedy sea shore and a fish market. The taste is a bit on the thin side but smooth too with dark roasted to burnt malts, chilli pepper notes, full up fish market counter oyster juice and earthiness. In the background is a mild sense of caramel. The fish/oyster sensation works well here with the earthy hops and roasty malts. Nice.'
 'Tasted on 8/1/11 from a 12oz bottle obtained in a trade, it was a bit of a gusher.  Pours thick, o

In [14]:
#Testes

for i in range(10):
    print(D_train['00000' + str(i)])
    print("yFeel:", y1Feel[i])
    print("yLook:", y1Look[i])
    print("ySmell:", y1Smell[i])
    print("yTaste:", y1Taste[i])
    print("yOverall:", y1Overall[i])
    print()

{'name': 'Anderson Valley Brother Davids Double', 'style': 'Abbey Dubbel', 'who': 'dmtroyer', 'feel': '3/5', 'look': '3/5', 'smell': '2/5', 'taste': '3/5', 'overall': '4/10', 'review': '22oz.  Pours dark brown with a red rim and off-white head.  Nose is mostly sweet malt with fairly noticeable alcohol depending on the temperature.  Flavor is sweet malt with raisin and banana yeast with some slight spice on the finish.  Very true to the style, but less than inspiring.'}
yFeel: 3
yLook: 3
ySmell: 2
yTaste: 3
yOverall: 4

{'name': 'Red Brick Double Chocolate Oatmeal Porter', 'style': 'Porter', 'who': 'Cornboy', 'feel': '4/5', 'look': '4/5', 'smell': '4/5', 'taste': '3/5', 'overall': '6/10', 'review': 'Compliments of a trade with goldtwins, thanks!\tPoured dark with very little head.  Coffee bean and chocolate dominate from the start, slight sweetness in the middle and finishes with a bit of bitterness from the coffee.  A nice hefty porter for the winter months.'}
yFeel: 4
yLook: 4
ySmell:

### Classificação Binária

A ideia é saber quais são as criticas que possuem uma classificação "overall" acima de 9 e abaixo de 2.<br>
Passos a seguir:
- Assim, temos de obter as criticas e classificações que pertencem a essa especificação.
- Criar dicionário para esta classificação. Este pode ser mais pequeno que o suposto para a classificação multi-classe.
- Pré-processar os dados. Limpar as criticas de caracteres HTML ou semelhantes e remover palavras "inválidas".
- Usar o tf-idf para vectorizar as palavras em cada crítica.
    - Deveria existir um stem neste ponto. Para uma primeira iteração, não o vamos fazer para termos uma melhor compreensão nas palavras (dimensões) que o tf-idf está a considerar importantes para classificar as criticas. Numa segunda iteração, após analisarmos os resultados, fazemos o Stemming.
- Usar o LogisticRegression para classificar as criticas.

In [15]:
#Treino
idx1 = (y1Overall < 3) | (y1Overall > 8)

X_Train_BinClass = X1[idx1]
y_Train_BinClass = y1Overall[idx1]

#Teste
idx2 = (y2Overall < 3) | (y2Overall > 8)

X_Test_BinClass = X2[idx2]
y_Test_BinClass = y2Overall[idx2]


print(X_Train_BinClass.shape)
print(y_Train_BinClass.shape)
print(X_Test_BinClass.shape)
print(y_Test_BinClass.shape)

(8398,)
(8398,)
(3540,)
(3540,)


In [16]:
#Treino
Docs1 = [doc.replace('<br />', ' ') for doc in X_Train_BinClass]
Docs1 = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1]

#Teste
Docs2 = [doc.replace('<br />', ' ') for doc in X_Test_BinClass]
Docs2 = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs2]

###### Para 3 letras ou mais

In [17]:
tfidf = TfidfVectorizer(min_df = 5, token_pattern = r'\b\w\w\w+\b').fit(Docs1)
tokens = tfidf.get_feature_names()

X_Train_tfidf = tfidf.transform(Docs1)
X_Test_tfidf = tfidf.transform(Docs2)

Para esta primeira iteração, ainda não temos em conta dados de validação, embora este vá ser necessário.<br>
Por agora, usamos os dados todos

In [43]:
print(len(tokens))

5357


In [18]:
#Palav
xM = np.max(X_Train_tfidf, axis=0).toarray().squeeze()
idx = np.argsort(-xM)
voc_ClassBin = [tokens[i] for i in idx]

#Organizar pelo idf, isto é, as palavras que aparecem mais vezes (menos importantes para deerminar a classificacao)
idf = tfidf.idf_
idx_idf = np.argsort(idf)
voc_ClassBin_idf = [tokens[i] for i in idx_idf]

In [19]:
print("Palavras mais Importantes:")
print(voc_ClassBin[:50])
print()
print("//-----------------//-----------------//-----------------//")
print()
print("Palavras menos Importantes:")
print(voc_ClassBin[-50:])
print()
print("//-----------------//-----------------//-----------------//")
print()
print("Palavras mais Frequentes:")
print(voc_ClassBin_idf[:50])

Palavras mais Importantes:
['des', 'men', 'molto', 'der', 'cedar', 'dans', 'lobster', 'blueberries', 'yum', 'peach', 'sapore', 'kiss', 'med', 'nitro', 'coke', 'pumpkin', 'nougat', 'munich', 'moderately', 'blueberry', 'why', 'yeast', 'menthol', 'summer', 'skunk', 'peu', 'non', 'crispy', 'cat', 'che', 'coffee', 'ringwood', 'est', 'ginger', 'tons', 'lightly', 'chalky', 'con', 'apricot', 'wild', 'raw', 'les', 'coriander', 'westy', 'peppermint', 'big', 'unpleasant', 'slight', 'tangy', 'herbs']

//-----------------//-----------------//-----------------//

Palavras menos Importantes:
['personal', 'warning', 'goddamn', 'vapors', 'beneath', 'praise', 'sublime', 'cornucopia', 'genius', 'halo', 'flood', 'brittle', 'grinds', 'cloudy', 'grade', 'ingredient', 'estimated', 'necessarily', 'popping', 'head', 'declare', 'splendid', 'seat', 'settling', 'precisely', 'lurking', 'resemblance', 'melded', 'lurks', 'depths', 'experiences', 'receded', 'steps', 'elegance', 'accentuate', 'sprinkled', 'picks', 'ha

##### Passar à Classificação das Criticas

In [20]:
#Forçar hiperparâmetros
dl = LogisticRegression(penalty = 'l2', max_iter = 1000, C = 10, tol = 1e-3)
dl.fit(X_Train_tfidf, y_Train_BinClass)

print(dl.score(X_Train_tfidf, y_Train_BinClass))
print(dl.score(X_Test_tfidf, y_Test_BinClass))

0.9339128363896165
0.5932203389830508


In [21]:
w = dl.coef_
idx = np.argsort(w).squeeze()

voc = tfidf.get_feature_names()

print(np.unique(y_Train_BinClass))
print(idx.shape)
voc2 = [voc[i] for i in idx]

print("Pesos com menor valor:")
print(voc[:50])
print()
print("//-----------------//-----------------//-----------------//")
print()
print("Pesos com maior valor:")
print(voc[-50:])

[1 2 9]
(3, 5357)


TypeError: only integer scalar arrays can be converted to a scalar index

Temos 3 classes no y_Train_BinClass em vez de 2: muito maus, muito bons.<br>
Isto leva a um erro <b>ESTUDAR O ERRO</b>.

Temos de binarizar os dados. Acima de 5 é muito bom, abaixo é muito mau (-1, 1)

In [22]:
y_Train_BinClass = np.where(y_Train_BinClass < 5, -1, 1)

print(np.unique(y_Train_BinClass))

[-1  1]


In [23]:
print("Número total de Cervejas: ", y_Train_BinClass.shape[0])
print("Cervejas muito Boas: ", np.sum(y_Train_BinClass > 0))
print("Cervejas muito Más: ", np.sum(y_Train_BinClass < 0))

Número total de Cervejas:  8398
Cervejas muito Boas:  3775
Cervejas muito Más:  4623


Vamos treinar novamente o classificador, mas antes disso, binarizar as classes para os dados de teste.

In [24]:
y_Test_BinClass = np.where(y_Test_BinClass < 5, -1, 1)

In [25]:
#Forçar hiperparâmetros
dl = LogisticRegression(penalty = 'l2', max_iter = 1000, C = 10, tol = 1e-3)
dl.fit(X_Train_tfidf, y_Train_BinClass)

print(dl.score(X_Train_tfidf, y_Train_BinClass))
print(dl.score(X_Test_tfidf, y_Test_BinClass))

0.93343653250774
0.8573446327683616


In [26]:
w = dl.coef_
idx = np.argsort(w).squeeze()

voc = tfidf.get_feature_names()
voc2 = [voc[i] for i in idx]

print("Pesos com menor valor:")
print(voc[:50])
print()
print("//-----------------//-----------------//-----------------//")
print()
print("Pesos com maior valor:")
print(voc[-50:])

Pesos com menor valor:
['abbey', 'ability', 'abita', 'able', 'abomination', 'abound', 'about', 'above', 'abrasive', 'abrupt', 'absent', 'absolute', 'absolutely', 'abt', 'abundance', 'abundant', 'abv', 'abyss', 'accent', 'accented', 'accents', 'accentuate', 'accentuated', 'acceptable', 'accepted', 'accessible', 'accompanied', 'accompany', 'accompanying', 'according', 'account', 'accurate', 'accurately', 'acetaldehyde', 'acetic', 'acetone', 'achieve', 'achieved', 'acid', 'acidic', 'acidity', 'acquired', 'acre', 'acrid', 'across', 'act', 'action', 'active', 'activity', 'actual']

//-----------------//-----------------//-----------------//

Pesos com maior valor:
['wreck', 'write', 'writing', 'wrong', 'wrote', 'wtf', 'wtih', 'www', 'yard', 'yeah', 'year', 'years', 'yeast', 'yeastiness', 'yeasts', 'yeasty', 'yellow', 'yellowish', 'yep', 'yer', 'yes', 'yespr', 'yesterday', 'yet', 'yields', 'yogurt', 'york', 'you', 'youd', 'youll', 'young', 'younger', 'your', 'youre', 'yourself', 'youth', 'yo

Com a primeira iteração efectuada, vamos passar ao desenvolvimento mais rigoroso desta classificação binária.

### Classificação Binária: Segunda Iteração 

In [27]:
#Obtenção das Criticas com Overall menor que 3 e maior que 8, para dados de Treino e dados de Teste
idx1 = (y1Overall < 3) | (y1Overall > 8)
X_Train_ClassBin = X1[idx1]
y_Train_ClassBin = y1Overall[idx1]

idx2 = (y2Overall < 3) | (y2Overall > 8)
X_Test_ClassBin = X2[idx2]
y_Test_ClassBin = y2Overall[idx2]

#Binarização das Classes
y_Train_ClassBin = np.where(y_Train_ClassBin < 5, -1, 1)
y_Test_ClassBin = np.where(y_Test_ClassBin < 5, -1, 1)

In [28]:
print(X_Train_ClassBin.shape)
print(y_Train_ClassBin.shape)
print()
print(X_Test_ClassBin.shape)
print(y_Test_ClassBin.shape)
print()
print(np.unique(y_Train_ClassBin))
print(np.unique(y_Test_ClassBin))

(8398,)
(8398,)

(3540,)
(3540,)

[-1  1]
[-1  1]


In [29]:
#Divisão dos Dados de Treino em Dados de Treino e Dados de Validação
Xa_Train_ClassBin, Xb_Train_ClassBin, ya_Train_ClassBin, yb_Train_ClassBin = train_test_split(X_Train_ClassBin, y_Train_ClassBin, test_size=1/3, stratify=y_Train_ClassBin, random_state=42)

In [30]:
print(Xa_Train_ClassBin.shape)
print(ya_Train_ClassBin.shape)
print()
print(Xb_Train_ClassBin.shape)
print(yb_Train_ClassBin.shape)
print()
print(np.unique(ya_Train_ClassBin))
print(np.unique(yb_Train_ClassBin))

(5598,)
(5598,)

(2800,)
(2800,)

[-1  1]
[-1  1]


In [50]:
#Criação do Dicionário
Docs1 = [doc.replace('<br />', ' ') for doc in X_Train_ClassBin]
Docs1 = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1]

Docs1a = [doc.replace('<br />', ' ') for doc in Xa_Train_ClassBin]
Docs1a = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1a]

Docs1b = [doc.replace('<br />', ' ') for doc in Xb_Train_ClassBin]
Docs1b = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1b]

#Teste
Docs2 = [doc.replace('<br />', ' ') for doc in X_Test_ClassBin]
Docs2 = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs2]

In [51]:
print(len(Docs1))
print(len(Docs1a))
print(len(Docs1b))
print(len(Docs2))

8398
5598
2800
3540


In [52]:
#Stemming
porterStemFunc = PorterStemmer()
Docs1_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs1a_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1a]
Docs1b_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1b]
Docs2_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs2]

snowballStemFunc = SnowballStemmer(language='english')
Docs1_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs1a_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1a]
Docs1b_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1b]
Docs2_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs2]

lancasterStemFunc = LancasterStemmer()
Docs1_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs1a_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1a]
Docs1b_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1b]
Docs2_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs2]

In [53]:
print(Docs1a_PorterStem[1])
print()
print(Docs1a_SnowballStem[1])
print()
print(Docs1a_LancasterStem[1])

oz nose is prune dark grape raisin junip and molass clear deep mahogani with a scarlet tint and a small film of last bubbl that leav a sugari lace tast of the same nice mix as the aroma plu chocol some more earthi spici and citru peel mint evergreen roasti char and woodi hop note make for a satisfyingli dri finish medium bodi is oili and wateri at the same time veri drinkabl for a sip beer malti sweet on front but great balanc from the roast hop and spici on the back end

oz nose is prune dark grape raisin junip and molass clear deep mahogani with a scarlet tint and a small film of last bubbl that leav a sugari lace tast of the same nice mix as the aroma plus chocol some more earthi spici and citrus peel mint evergreen roasti char and woodi hop note make for a satisfi dri finish medium bodi is oili and wateri at the same time veri drinkabl for a sip beer malti sweet on front but great balanc from the roast hop and spici on the back end

oz nos is prun dark grap raisin junip and molass 

In [ ]:
#tf-idf de cada stemmer
tfidf_porter = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w+\b').fit(Docs1a_PorterStem)
X1_porter = tfidf_porter.transform(Docs1_PorterStem)
X1a_porter = tfidf_porter.transform(Docs1a_PorterStem)
X1b_porter = tfidf_porter.transform(Docs1b_PorterStem)
X2_porter = tfidf_porter.transform(Docs2_PorterStem)
tokens_porter = tfidf_porter.get_feature_names()

tfidf_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w+\b').fit(Docs1a_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_snowball = tfidf_snowball.transform(Docs1a_SnowballStem)
X1b_snowball = tfidf_snowball.transform(Docs1b_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

tfidf_lancaster = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w+\b').fit(Docs1a_LancasterStem)
X1_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_lancaster = tfidf_lancaster.transform(Docs1a_LancasterStem)
X1b_lancaster = tfidf_lancaster.transform(Docs1b_LancasterStem)
X2_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_lancaster = tfidf_lancaster.get_feature_names()

In [55]:
print(len(tokens_porter))
print(len(tokens_snowball))
print(len(tokens_lancaster))

4738
4676
4130


Podemos observar uma ligeira diminuição no número de palavras que permanecem nos dicionários, após o stemming.<br>
Podemos alterar os valores do tf-idf, mas por agora vamos manter estes resultados.

In [57]:
#Forçar hiperparâmetros
dl_porter = LogisticRegression(penalty = 'l2', max_iter = 1000, C = 10, tol = 1e-3)
dl_porter.fit(X1a_porter, ya_Train_ClassBin)

print(dl_porter.score(X1a_porter, ya_Train_ClassBin))
print(dl_porter.score(X1b_porter, yb_Train_ClassBin))
print()


dl_snowball = LogisticRegression(penalty = 'l2', max_iter = 1000, C = 10, tol = 1e-3)
dl_snowball.fit(X1a_snowball, ya_Train_ClassBin)

print(dl_snowball.score(X1a_snowball, ya_Train_ClassBin))
print(dl_snowball.score(X1b_snowball, yb_Train_ClassBin))
print()


dl_lancaster = LogisticRegression(penalty = 'l2', max_iter = 1000, C = 10, tol = 1e-3)
dl_lancaster.fit(X1a_lancaster, ya_Train_ClassBin)

print(dl_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print(dl_lancaster.score(X1b_lancaster, yb_Train_ClassBin))

0.9387281171847088
0.7810714285714285

0.938549481957842
0.7825

0.9271168274383709
0.7782142857142857


Todos os stemmers oferecem a mesma descrepância na classificação entre os dados de treino e validação.<br>
Vamos treinar de novo o classificador, mas desta vez com a regularização Lasso (l1).

In [59]:
#Forçar hiperparâmetros
dl_porter = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 5000, C = 10)
dl_porter.fit(X1a_porter, ya_Train_ClassBin)

print(dl_porter.score(X1a_porter, ya_Train_ClassBin))
print(dl_porter.score(X1b_porter, yb_Train_ClassBin))
print()


dl_snowball = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 5000, C = 10)
dl_snowball.fit(X1a_snowball, ya_Train_ClassBin)

print(dl_snowball.score(X1a_snowball, ya_Train_ClassBin))
print(dl_snowball.score(X1b_snowball, yb_Train_ClassBin))
print()


dl_lancaster = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 5000, C = 10)
dl_lancaster.fit(X1a_lancaster, ya_Train_ClassBin)

print(dl_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print(dl_lancaster.score(X1b_lancaster, yb_Train_ClassBin))

0.9773133261879242
0.7460714285714286

0.9757056091461236
0.7485714285714286

0.9660593068953197
0.7492857142857143


As máximas iterações tiveram de ser mudadas dado que era lançado um aviso que não foi possível atingir a convergência com 1000 iterações.

Dados de Treino têm uma muito melhor classificação com o Lasso do que com o Ridge.<br>
Vamos usar Máquinas de Suporte Vectorial (SVM).

## SVMs

### SVM - PorterStemmer

In [77]:
begin = time.time()

topScore = 0
ListG = [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.01, 0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListG:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  110 . G =  0.001  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.34879994392395
1  -  110 . G =  0.005  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.129854917526245
2  -  110 . G =  0.01  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.00182843208313
3  -  110 . G =  0.05  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.661861181259155
4  -  110 . G =  0.1  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:28.59909963607788
5  -  110 . G =  0.25  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:27.550422430038452
6  -  110 . G =  0.5  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.081846714019775
7  -  110 . G =  0.75  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.093998908996582
8  -  110 . G =  1  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.055250883102417
9  -  110 . G =  auto  | C =  0.001  | Score: 0.5503571428571429  | Time:  0:26.10231900215149
10  -  110 . G =  0.001  | C =  0.005  | Score:

87  -  110 . G =  0.75  | C =  1  | Score: 0.8039285714285714  | Time:  0:21.170419692993164
88  -  110 . G =  1  | C =  1  | Score: 0.8021428571428572  | Time:  0:22.922125339508057
89  -  110 . G =  auto  | C =  1  | Score: 0.5503571428571429  | Time:  0:26.260371208190918


TypeError: must be real number, not str

Aparentemente deu erro quando o g seria 0.001 e o C seria 2.5<br>
Vamos isolar esse caso e ver o que se passa

In [79]:
svm_porter = SVC(kernel = 'rbf', gamma = 0.001, C = 2.5).fit(X1a_porter, ya_Train_ClassBin)
score = svm_porter.score(X1b_porter, yb_Train_ClassBin)

print(score)

0.5503571428571429


Isolado funciona. Não consigo perceber o que pode ter acontecido. <b>Perguntar ao engenheiro</b>.<br>
No entanto, podemos concluir, analisando os resultados, que os valores de C iguais a 0.001, 0.005, 0.01 e até 0.05 têm maus resultados. A partir do 0.1 já começam a ter melhores resultados, embora sejam baixos na mesma.
Para o caso do gama, os valores 0.001, 0.005 e até 0.01 têm maus desempenhos. A partir do 0.05 já começam a ser melhores.<br>
O C toma estes valores porque o c que o for está a obter vem da lista dos G's. Isto foi claramente um erro que é facilmente corrigido e, depois de analisados os resultados, deve ser feito.

Assim, de forma a poupar tempo e evitar operações desnecessárias, vamos repetir o processo feito acima, mas desta vez com a lista dos C's a começar em 0.1 e dos gamas a começar em 0.05.

In [80]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:27.952532291412354
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5939285714285715  | Time:  0:27.63221549987793
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7203571428571428  | Time:  0:28.554036140441895
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7528571428571429  | Time:  0:26.944632053375244
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7535714285714286  | Time:  0:25.389103412628174
5  -  70 . G =  1  | C =  0.1  | Score: 0.7275  | Time:  0:26.038859605789185
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:27.013277530670166
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7635714285714286  | Time:  0:25.593796253204346
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7964285714285714  | Time:  0:23.102718830108643
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.8039285714285714  | Time:  0:21.285670280456543
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.8035714285714286  | Time:  0:21.123495578765

In [89]:
#Gravar dados até agora.

#Vocabulario com PorterStemmer:
Docs_PorterStem = {
    "Docs1_PorterStem" : Docs1_PorterStem,
    "Docs1a_PorterStem" : Docs1a_PorterStem,
    "Docs1b_PorterStem" : Docs1b_PorterStem,
    "Docs2_PorterStem" : Docs2_PorterStem
}

Docs_SnowballStem = {
    "Docs1_SnowballStem" : Docs1_SnowballStem,
    "Docs1a_SnowballStem" : Docs1a_SnowballStem,
    "Docs1b_SnowballStem" : Docs1b_SnowballStem,
    "Docs2_SnowballStem" : Docs2_SnowballStem
}

Docs_LancasterStem = {
    "Docs1_LancasterStem" : Docs1_LancasterStem,
    "Docs1a_LancasterStem" : Docs1a_LancasterStem,
    "Docs1b_LancasterStem" : Docs1b_LancasterStem,
    "Docs2_LancasterStem" : Docs2_LancasterStem
}

tfidf_porter_dict = {
    "tfidf_porter" : tfidf_porter,
    "X1_porter" : X1_porter,
    "X1a_porter" : X1a_porter,
    "X1b_porter" : X1b_porter,
    "X2_porter" : X2_porter,
    "tokens_porter" : tokens_porter
}

tfidf_snowball_dict = {
    "tfidf_snowball" : tfidf_snowball,
    "X1_snowball" : X1_snowball,
    "X1a_snowball" : X1a_snowball,
    "X1b_snowball" : X1b_snowball,
    "X2_snowball" : X2_snowball,
    "tokens_snowball" : tokens_snowball
}

tfidf_lancaster_dict = {
    "tfidf_lancaster" : tfidf_lancaster,
    "X1_lancaster" : X1_lancaster,
    "X1a_lancaster" : X1a_lancaster,
    "X1b_lancaster" : X1b_lancaster,
    "X2_lancaster" : X2_lancaster,
    "tokens_lancaster" : tokens_lancaster
}

logisticRegression_Lasso = {
    "dl_porter" : dl_porter,
    "dl_snowball" : dl_snowball,
    "dl_lancaster" : dl_lancaster
}

svm_porter_dict = {
    "svm_porter" : svm_porter
}


with open('Dados/dados_14_12_2023_Docs_PorterStem.p', 'wb') as handle:
    pickle.dump(Docs_PorterStem, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Dados/dados_14_12_2023_Docs_SnowballStem.p', 'wb') as handle:
    pickle.dump(Docs_SnowballStem, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Dados/dados_14_12_2023_Docs_LancasterStem.p', 'wb') as handle:
    pickle.dump(Docs_LancasterStem, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Dados/dados_14_12_2023_tfidf_porter_dict.p', 'wb') as handle:
    pickle.dump(tfidf_porter_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Dados/dados_14_12_2023_tfidf_snowball_dict.p', 'wb') as handle:
    pickle.dump(tfidf_snowball_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('Dados/dados_14_12_2023_tfidf_lancaster_dict.p', 'wb') as handle:
    pickle.dump(tfidf_lancaster_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Dados/dados_14_12_2023_logisticRegression_Lasso.p', 'wb') as handle:
    pickle.dump(logisticRegression_Lasso, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Dados/dados_14_12_2023_svm_porter_dict.p', 'wb') as handle:
    pickle.dump(svm_porter_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [91]:
#Testar Leitura de Dados
print(pickle.load(open("Dados/dados_14_12_2023_Docs_PorterStem.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_Docs_SnowballStem.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_Docs_LancasterStem.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_tfidf_porter_dict.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_tfidf_snowball_dict.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_tfidf_lancaster_dict.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_logisticRegression_Lasso.p", 'rb')).keys())
print(pickle.load(open("Dados/dados_14_12_2023_svm_porter_dict.p", 'rb')).keys())

dict_keys(['Docs1_PorterStem', 'Docs1a_PorterStem', 'Docs1b_PorterStem', 'Docs2_PorterStem'])
dict_keys(['Docs1_SnowballStem', 'Docs1a_SnowballStem', 'Docs1b_SnowballStem', 'Docs2_SnowballStem'])
dict_keys(['Docs1_LancasterStem', 'Docs1a_LancasterStem', 'Docs1b_LancasterStem', 'Docs2_LancasterStem'])
dict_keys(['tfidf_porter', 'X1_porter', 'X1a_porter', 'X1b_porter', 'X2_porter', 'tokens_porter'])
dict_keys(['tfidf_snowball', 'X1_snowball', 'X1a_snowball', 'X1b_snowball', 'X2_snowball', 'tokens_snowball'])
dict_keys(['tfidf_lancaster', 'X1_lancaster', 'X1a_lancaster', 'X1b_lancaster', 'X2_lancaster', 'tokens_lancaster'])
dict_keys(['dl_porter', 'dl_snowball', 'dl_lancaster'])
dict_keys(['svm_porter'])


Repetir o mesmo processo de SVM acima para os dados com o SnowballStemmer

### SVM - SnowballStemmer

In [94]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:27.40352702140808
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5925  | Time:  0:27.883336544036865
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7232142857142857  | Time:  0:27.941513776779175
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7582142857142857  | Time:  0:26.916019439697266
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7521428571428571  | Time:  0:29.421569108963013
5  -  70 . G =  1  | C =  0.1  | Score: 0.7307142857142858  | Time:  0:28.205331563949585
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:26.352597951889038
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7675  | Time:  0:25.97511649131775
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7960714285714285  | Time:  0:23.709391117095947
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.8042857142857143  | Time:  0:21.909006595611572
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.8035714285714286  | Time:  0:21.30783247947693
11  -  70 

Mesmo processo acima, mas para os dados com o LancasterStemmer

### SVM - LancasterStemmer

In [95]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:25.77813959121704
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5942857142857143  | Time:  0:25.722564458847046
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7285714285714285  | Time:  0:25.11636996269226
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7521428571428571  | Time:  0:24.935511112213135
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7510714285714286  | Time:  0:25.106693744659424
5  -  70 . G =  1  | C =  0.1  | Score: 0.7346428571428572  | Time:  0:25.26062321662903
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:25.7295503616333
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7664285714285715  | Time:  0:24.24729609489441
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7939285714285714  | Time:  0:22.744107007980347
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7978571428571428  | Time:  0:20.676311492919922
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.7982142857142858  | Time:  0:20.76372

Nenhum dos scores é propriamente fantástico. Ainda podemos usar os n-gramas e até modificar o token-pattern do TfidfVectorizer.<br>
Antes disso, vamos repetir a classificação do SVM, mas sem stemmer e analisar o resultado.

In [97]:
tfidf_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w+\b').fit(Docs1a)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:31.286524295806885
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5675  | Time:  0:26.902287006378174
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7  | Time:  0:26.685338020324707
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7296428571428571  | Time:  0:26.295046091079712
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7253571428571428  | Time:  0:26.796916723251343
5  -  70 . G =  1  | C =  0.1  | Score: 0.7046428571428571  | Time:  0:26.94391179084778
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:26.622249603271484
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7482142857142857  | Time:  0:25.860594749450684
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7867857142857143  | Time:  0:24.252902269363403
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7964285714285714  | Time:  0:22.51416778564453
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.8003571428571429  | Time:  0:22.147791385650635
11  -  70 . 

Conclusões tiradas nas classificações com SVM:
- Sem stemming:
    - Treino: 87.3%
    - Validação: 80.3%
- Porter Stemming:
    - Treino: 90.5%
    - Validação: 80.5%
- Snowball Stemming:
    - Treino: 93.3%
    - Validação: 80.6%
- Lancaster Stemming:
    - Treino: 95.4%
    - Validação: 80.1%

Ao interpretar estes dados, concluimos que o SVM desempenha uma melhor classificação se o treinarmos com um dicionário pré-processado com o Snowball Stemmer, embora seja apenas por uma décima percentual.

Vamos então modificar os hiper-parametros do TfidfVectorizer.

In [115]:
tfidf_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a)
X1_noStem = tfidf_noStem.transform(Docs1)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
X2_noStem = tfidf_noStem.transform(Docs2)
tokens_noStem = tfidf_noStem.get_feature_names()

tfidf_porter = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a_PorterStem)
X1_porter = tfidf_porter.transform(Docs1_PorterStem)
X1a_porter = tfidf_porter.transform(Docs1a_PorterStem)
X1b_porter = tfidf_porter.transform(Docs1b_PorterStem)
X2_porter = tfidf_porter.transform(Docs2_PorterStem)
tokens_porter = tfidf_porter.get_feature_names()

tfidf_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_snowball = tfidf_snowball.transform(Docs1a_SnowballStem)
X1b_snowball = tfidf_snowball.transform(Docs1b_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

tfidf_lancaster = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a_LancasterStem)
X1_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_lancaster = tfidf_lancaster.transform(Docs1a_LancasterStem)
X1b_lancaster = tfidf_lancaster.transform(Docs1b_LancasterStem)
X2_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_lancaster = tfidf_lancaster.get_feature_names()

In [116]:
print(len(tokens_noStem))
print(len(tokens_porter))
print(len(tokens_snowball))
print(len(tokens_lancaster))

6175
4885
4813
4292


### SVM - No Stem

In [101]:
topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:30.336706399917603
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5671428571428572  | Time:  0:30.029898166656494
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.6975  | Time:  0:29.885671615600586
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.7253571428571428  | Time:  0:29.465953826904297
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.7207142857142858  | Time:  0:29.566771984100342
6  -  70 . G =  1  | C =  0.1  | Score: 0.7021428571428572  | Time:  0:30.286702394485474
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:29.78152561187744
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.7410714285714286  | Time:  0:28.93693447113037
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7867857142857143  | Time:  0:27.35104990005493
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7925  | Time:  0:25.821799278259277
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.7957142857142857  | Time:  0:26.605186223983765
12  -  70

### SVM - PorterStemmer

In [103]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:30.04836106300354
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5828571428571429  | Time:  0:29.734002828598022
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7139285714285715  | Time:  0:29.483662605285645
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7510714285714286  | Time:  0:28.644705772399902
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7453571428571428  | Time:  0:28.839227199554443
5  -  70 . G =  1  | C =  0.1  | Score: 0.7246428571428571  | Time:  0:29.637850761413574
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:29.628220558166504
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7610714285714286  | Time:  0:28.25298500061035
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7946428571428571  | Time:  0:26.246077060699463
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7982142857142858  | Time:  0:24.378481149673462
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.8025  | Time:  0:23.9857556819915

### SVM - Snowball Stemmer

In [104]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:31.667505979537964
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5860714285714286  | Time:  0:30.094491720199585
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7185714285714285  | Time:  0:29.485716104507446
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7503571428571428  | Time:  0:28.548973560333252
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7475  | Time:  0:28.814427137374878
5  -  70 . G =  1  | C =  0.1  | Score: 0.7239285714285715  | Time:  0:29.328521966934204
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:29.461896657943726
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7664285714285715  | Time:  0:28.21212100982666
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7953571428571429  | Time:  0:26.171759843826294
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7996428571428571  | Time:  0:24.18807363510132
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.8010714285714285  | Time:  0:24.3491067886352

In [105]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:30.4493670463562
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5885714285714285  | Time:  0:29.63354754447937
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7246428571428571  | Time:  0:29.727430820465088
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7471428571428571  | Time:  0:31.57481360435486
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7467857142857143  | Time:  0:29.63385510444641
5  -  70 . G =  1  | C =  0.1  | Score: 0.7321428571428571  | Time:  0:29.38240623474121
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:29.597312688827515
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7632142857142857  | Time:  0:28.10129737854004
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7971428571428572  | Time:  0:26.034929037094116
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7975  | Time:  0:23.97962999343872
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.8  | Time:  0:23.597655296325684
11  -  70 . G =  0

Conclusões tiradas nas classificações com SVM:
- Sem stemming:
    - Treino: 96.2%
    - Validação: 80.0%
- Porter Stemming:
    - Treino: 88.8%
    - Validação: 80.4%
- Snowball Stemming:
    - Treino: 90.2%
    - Validação: 80.1%
- Lancaster Stemming:
    - Treino: 86.5%
    - Validação: 80.0%

Para este caso, é necessário lembrar que o TfidfVectorizer aceita um token pattern com, no mínimo, dois caracteres.<br>
Ao interpretar estes dados, concluimos que o SVM desempenha uma melhor classificação se o treinarmos com um dicionário pré-processado com o Porter Stemmer, embora seja apenas por três décimas percentuais.

Vamos então modificar os hiper-parametros do TfidfVectorizer para aceitar quatro ou mais caracteres.<br>
Isto vai tornar os dicionários mais limitados. Isto implica menos dados de treino o que certamente influencia a classificação do conjunto de validação e de teste.

In [117]:
tfidf_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w\w+\b').fit(Docs1a)
X1_noStem = tfidf_noStem.transform(Docs1)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
X2_noStem = tfidf_noStem.transform(Docs2)
tokens_noStem = tfidf_noStem.get_feature_names()

tfidf_porter = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w\w+\b').fit(Docs1a_PorterStem)
X1_porter = tfidf_porter.transform(Docs1_PorterStem)
X1a_porter = tfidf_porter.transform(Docs1a_PorterStem)
X1b_porter = tfidf_porter.transform(Docs1b_PorterStem)
X2_porter = tfidf_porter.transform(Docs2_PorterStem)
tokens_porter = tfidf_porter.get_feature_names()

tfidf_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w\w+\b').fit(Docs1a_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_snowball = tfidf_snowball.transform(Docs1a_SnowballStem)
X1b_snowball = tfidf_snowball.transform(Docs1b_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

tfidf_lancaster = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w\w+\b').fit(Docs1a_LancasterStem)
X1_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_lancaster = tfidf_lancaster.transform(Docs1a_LancasterStem)
X1b_lancaster = tfidf_lancaster.transform(Docs1b_LancasterStem)
X2_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_lancaster = tfidf_lancaster.get_feature_names()

In [118]:
print(len(tokens_noStem))
print(len(tokens_porter))
print(len(tokens_snowball))
print(len(tokens_lancaster))

5666
4326
4256
3391


### SVM - No Stem

In [110]:
topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:21.50681209564209
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5657142857142857  | Time:  0:23.17811608314514
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.6892857142857143  | Time:  0:22.72335648536682
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.72  | Time:  0:22.433940649032593
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.7107142857142857  | Time:  0:22.411843299865723
6  -  70 . G =  1  | C =  0.1  | Score: 0.6921428571428572  | Time:  0:24.279875993728638
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:22.725592374801636
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.7378571428571429  | Time:  0:21.94647240638733
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7732142857142857  | Time:  0:20.53557276725769
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7882142857142858  | Time:  0:18.923027753829956
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.795  | Time:  0:18.769891023635864
12  -  70 . G 

### SVM - Porter Stemmer

In [111]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:21.548243761062622
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5889285714285715  | Time:  0:21.714495420455933
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7082142857142857  | Time:  0:21.152534008026123
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7385714285714285  | Time:  0:20.681121587753296
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7325  | Time:  0:21.260378122329712
5  -  70 . G =  1  | C =  0.1  | Score: 0.7114285714285714  | Time:  0:21.71174383163452
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:21.56618309020996
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7553571428571428  | Time:  0:20.329842805862427
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7785714285714286  | Time:  0:18.911989212036133
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7903571428571429  | Time:  0:17.55166220664978
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.7921428571428571  | Time:  0:17.54562139511108

### SVM - Snowball Stemmer

In [112]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:24.843857526779175
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5896428571428571  | Time:  0:22.874774932861328
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.7075  | Time:  0:21.796988487243652
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7378571428571429  | Time:  0:21.151674032211304
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7335714285714285  | Time:  0:21.43269658088684
5  -  70 . G =  1  | C =  0.1  | Score: 0.7110714285714286  | Time:  0:21.809898138046265
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:21.968137979507446
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7553571428571428  | Time:  0:20.868281602859497
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7796428571428572  | Time:  0:19.396379947662354
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7921428571428571  | Time:  0:18.14035415649414
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.7925  | Time:  0:17.780987977981567
11  -  70

### SVM - Lancaster Stemmer

In [113]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:17.72577691078186
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5971428571428572  | Time:  0:16.853628873825073
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.6946428571428571  | Time:  0:16.870430946350098
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.7242857142857143  | Time:  0:16.483699321746826
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.7167857142857142  | Time:  0:16.2789888381958
5  -  70 . G =  1  | C =  0.1  | Score: 0.7003571428571429  | Time:  0:17.20118260383606
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:16.826042890548706
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.7392857142857143  | Time:  0:16.05558490753174
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7685714285714286  | Time:  0:14.912829399108887
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7814285714285715  | Time:  0:13.764982461929321
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.7821428571428571  | Time:  0:13.7332

Conclusões tiradas nas classificações com SVM:
- Sem stemming:
    - Treino:  87.2%
    - Validação: 79.6%
- Porter Stemming:
    - Treino: 86.5%
    - Validação: 79.7%
- Snowball Stemming:
    - Treino: 95.5%
    - Validação: 79.6%
- Lancaster Stemming:
    - Treino: 89.7%
    - Validação: 78.4%

Ao interpretar estes dados, concluimos que o SVM desempenha uma melhor classificação se o treinarmos com um dicionário pré-processado com o Porter Stemmer, embora seja apenas por uma décima percentual.<br>
Outra clara conclusão ao compararmos com os testes efectuados previamente é que a restrição do "token_pattern" no TfidfVectorizer com quatro ou mais caracteres, proporciona resultados piores que um token_pattern de 2 ou mais caracteres, e que 3 ou mais. Assim sendo, vamos escolher um "token_pattern" com 2 ou mais caracteres e vamos passar a incluir "n-gramas".<br>

Vamos começar por aceitar bi-gramas e analisar os resultados.

In [119]:
tfidf_noStem = TfidfVectorizer(min_df = 3, ngram_range = (1, 2), token_pattern = r'\b\w\w+\b').fit(Docs1a)
X1_noStem = tfidf_noStem.transform(Docs1)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
X2_noStem = tfidf_noStem.transform(Docs2)
tokens_noStem = tfidf_noStem.get_feature_names()

tfidf_porter = TfidfVectorizer(min_df = 3, ngram_range = (1, 2), token_pattern = r'\b\w\w+\b').fit(Docs1a_PorterStem)
X1_porter = tfidf_porter.transform(Docs1_PorterStem)
X1a_porter = tfidf_porter.transform(Docs1a_PorterStem)
X1b_porter = tfidf_porter.transform(Docs1b_PorterStem)
X2_porter = tfidf_porter.transform(Docs2_PorterStem)
tokens_porter = tfidf_porter.get_feature_names()

tfidf_snowball = TfidfVectorizer(min_df = 3, ngram_range = (1, 2), token_pattern = r'\b\w\w+\b').fit(Docs1a_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_snowball = tfidf_snowball.transform(Docs1a_SnowballStem)
X1b_snowball = tfidf_snowball.transform(Docs1b_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

tfidf_lancaster = TfidfVectorizer(min_df = 3, ngram_range = (1, 2), token_pattern = r'\b\w\w+\b').fit(Docs1a_LancasterStem)
X1_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_lancaster = tfidf_lancaster.transform(Docs1a_LancasterStem)
X1b_lancaster = tfidf_lancaster.transform(Docs1b_LancasterStem)
X2_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_lancaster = tfidf_lancaster.get_feature_names()

In [120]:
print(len(tokens_noStem))
print(len(tokens_porter))
print(len(tokens_snowball))
print(len(tokens_lancaster))

30561
29675
29622
29316


### SVM - No Stem

In [122]:
topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:48.034302949905396
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:47.664417028427124
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.555  | Time:  0:47.6061851978302
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.5803571428571429  | Time:  0:47.61672401428223
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.5753571428571429  | Time:  0:47.936665534973145
6  -  70 . G =  1  | C =  0.1  | Score: 0.5557142857142857  | Time:  0:48.91458988189697
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:46.46542477607727
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.6171428571428571  | Time:  0:47.62940692901611
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7417857142857143  | Time:  0:46.137412786483765
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7903571428571429  | Time:  0:44.1653413772583
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.7957142857142857  | Time:  0:43.83651351928711
12  

### SVM - Porter Stemmer

In [123]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:51.04443383216858
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:48.44381093978882
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.5603571428571429  | Time:  0:48.3747501373291
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.5960714285714286  | Time:  0:48.8433997631073
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.5835714285714285  | Time:  0:49.5633602142334
5  -  70 . G =  1  | C =  0.1  | Score: 0.5607142857142857  | Time:  0:49.82234859466553
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:47.672701835632324
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.6325  | Time:  0:48.39103984832764
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7532142857142857  | Time:  0:47.10202622413635
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.795  | Time:  0:44.468801498413086
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.7957142857142857  | Time:  0:44.29500961303711
11  -  70 . G =  0.7

### SVM - Snowball Stemmer

In [127]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:48.64925265312195
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:48.00210666656494
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.5610714285714286  | Time:  0:48.04545211791992
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.5975  | Time:  0:48.69794726371765
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.5835714285714285  | Time:  0:48.69278597831726
6  -  70 . G =  1  | C =  0.1  | Score: 0.5614285714285714  | Time:  0:49.547380685806274
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:47.104312896728516
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.6346428571428572  | Time:  0:48.019675493240356
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7539285714285714  | Time:  0:46.418577432632446
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7925  | Time:  0:43.848859548568726
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.7957142857142857  | Time:  0:43.92562246322632
12  -  70 . 

### SVM - Lancaster Stemmer

In [128]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:49.310341358184814
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:49.433879137039185
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.5610714285714286  | Time:  0:50.45776963233948
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.5917857142857142  | Time:  0:51.1122100353241
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.5832142857142857  | Time:  0:51.41661858558655
6  -  70 . G =  1  | C =  0.1  | Score: 0.5607142857142857  | Time:  0:51.44725322723389
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:48.842079877853394
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.6317857142857143  | Time:  0:49.12240242958069
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7528571428571429  | Time:  0:49.131909132003784
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7914285714285715  | Time:  0:46.76787567138672
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.8007142857142857  | Time:  0:46.88821

Conclusões tiradas nas classificações com SVM:
- Sem stemming:
    - Treino:  98.5%
    - Validação: 80.3%
- Porter Stemming:
    - Treino: 96.8%
    - Validação: 80.6%
- Snowball Stemming:
    - Treino: 93.7%
    - Validação: 80.6%
- Lancaster Stemming:
    - Treino: 98.4%
    - Validação: 80.4%

Ao interpretar estes dados, concluimos que o SVM desempenha uma melhor classificação se o treinarmos com um dicionário pré-processado com o Porter Stemmer ou o Snowball Stemmer.<br>
Outra clara conclusão ao compararmos com os testes efectuados previamente é que o vocabulário a incluir bi-gramas é muito maior que apenas com unigramas, como era expectável. Apresentando uma lista de tamanhos para uma comparação facilitada:  

- No Stem:
    - Unigramas: 5666
    - Unigramas e Bi-gramas: 30561
- Porter Stemmer:
    - Unigramas: 4326
    - Unigramas e Bi-gramas: 29675
- Snowball Stemmer:
    - Unigramas: 4256
    - Unigramas e Bi-gramas: 29622
- Lancaster Stemmer:
    - Unigramas: 3391
    - Unigramas e Bi-gramas: 29316

É possível observar o aumento exponencial de um caso para outro. Em alguns casos, o vocabulário torna-se mais de oito vezes maior. Isto traduz-se no aumento considerado de tempo de treino e avaliação dos classificadores. 

Ainda assim, vamos incluir tri-gramas e observar os resultados

In [130]:
tfidf_noStem = TfidfVectorizer(min_df = 3, ngram_range = (1, 3), token_pattern = r'\b\w\w+\b').fit(Docs1a)
X1_noStem = tfidf_noStem.transform(Docs1)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
X2_noStem = tfidf_noStem.transform(Docs2)
tokens_noStem = tfidf_noStem.get_feature_names()

tfidf_porter = TfidfVectorizer(min_df = 3, ngram_range = (1, 3), token_pattern = r'\b\w\w+\b').fit(Docs1a_PorterStem)
X1_porter = tfidf_porter.transform(Docs1_PorterStem)
X1a_porter = tfidf_porter.transform(Docs1a_PorterStem)
X1b_porter = tfidf_porter.transform(Docs1b_PorterStem)
X2_porter = tfidf_porter.transform(Docs2_PorterStem)
tokens_porter = tfidf_porter.get_feature_names()

tfidf_snowball = TfidfVectorizer(min_df = 3, ngram_range = (1, 3), token_pattern = r'\b\w\w+\b').fit(Docs1a_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_snowball = tfidf_snowball.transform(Docs1a_SnowballStem)
X1b_snowball = tfidf_snowball.transform(Docs1b_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

tfidf_lancaster = TfidfVectorizer(min_df = 3, ngram_range = (1, 3), token_pattern = r'\b\w\w+\b').fit(Docs1a_LancasterStem)
X1_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_lancaster = tfidf_lancaster.transform(Docs1a_LancasterStem)
X1b_lancaster = tfidf_lancaster.transform(Docs1b_LancasterStem)
X2_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_lancaster = tfidf_lancaster.get_feature_names()

In [131]:
print(len(tokens_noStem))
print(len(tokens_porter))
print(len(tokens_snowball))
print(len(tokens_lancaster))

46556
47054
47066
47129


### SVM - No Stem

In [132]:
topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:55.03770399093628
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:55.69974946975708
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:55.38229489326477
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.5542857142857143  | Time:  0:53.42002630233765
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.5525  | Time:  0:53.90538811683655
6  -  70 . G =  1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:54.75218749046326
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:51.743064403533936
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.5710714285714286  | Time:  0:52.94329261779785
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7182142857142857  | Time:  0:53.392494678497314
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7846428571428572  | Time:  0:50.55161666870117
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.7925  | Time:  0:50.367738246917725
12  -  70 . G 

### SVM - Porter Stemmer

In [133]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:54.474196672439575
1  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:54.348506689071655
2  -  70 . G =  0.25  | C =  0.1  | Score: 0.5507142857142857  | Time:  0:54.58374047279358
3  -  70 . G =  0.5  | C =  0.1  | Score: 0.5567857142857143  | Time:  0:54.86548924446106
4  -  70 . G =  0.75  | C =  0.1  | Score: 0.5525  | Time:  0:55.61326885223389
5  -  70 . G =  1  | C =  0.1  | Score: 0.5510714285714285  | Time:  0:56.05026626586914
6  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:53.216389894485474
7  -  70 . G =  0.05  | C =  0.5  | Score: 0.5767857142857142  | Time:  0:54.352357149124146
8  -  70 . G =  0.1  | C =  0.5  | Score: 0.7278571428571429  | Time:  0:53.80768299102783
9  -  70 . G =  0.25  | C =  0.5  | Score: 0.7917857142857143  | Time:  0:51.31132793426514
10  -  70 . G =  0.5  | C =  0.5  | Score: 0.7946428571428571  | Time:  0:51.33856391906738
11

### SVM - Snowball Stemmer

In [134]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:57.23330307006836
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:54.25841569900513
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.5507142857142857  | Time:  0:54.47716927528381
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.5571428571428572  | Time:  0:55.693830490112305
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.5528571428571428  | Time:  0:55.56776285171509
6  -  70 . G =  1  | C =  0.1  | Score: 0.5507142857142857  | Time:  0:55.88859748840332
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:53.17733693122864
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.5771428571428572  | Time:  0:54.17198038101196
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.7289285714285715  | Time:  0:54.24286222457886
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7907142857142857  | Time:  0:51.7034125328064
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.7964285714285714  | Time:  0:51.25563764

### SVM - Lancaster Stemmer

In [135]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  70 . G =  0.05  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:55.5942223072052
2  -  70 . G =  0.1  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:54.54966425895691
3  -  70 . G =  0.25  | C =  0.1  | Score: 0.5510714285714285  | Time:  0:54.74875831604004
4  -  70 . G =  0.5  | C =  0.1  | Score: 0.5553571428571429  | Time:  0:55.31372022628784
5  -  70 . G =  0.75  | C =  0.1  | Score: 0.5532142857142858  | Time:  0:55.84456253051758
6  -  70 . G =  1  | C =  0.1  | Score: 0.5510714285714285  | Time:  0:56.15105628967285
7  -  70 . G =  auto  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:53.54506778717041
8  -  70 . G =  0.05  | C =  0.5  | Score: 0.5735714285714286  | Time:  0:54.607157707214355
9  -  70 . G =  0.1  | C =  0.5  | Score: 0.725  | Time:  0:54.15159511566162
10  -  70 . G =  0.25  | C =  0.5  | Score: 0.7896428571428571  | Time:  0:51.543827056884766
11  -  70 . G =  0.5  | C =  0.5  | Score: 0.7914285714285715  | Time:  0:51.64971423149109
12  -

Conclusões tiradas nas classificações com SVM:
- Sem stemming:
    - Treino:  95.0%
    - Validação: 80.3%
- Porter Stemming:
    - Treino: 90.8%
    - Validação: 80.5%
- Snowball Stemming:
    - Treino: 90.4%
    - Validação: 80.7%
- Lancaster Stemming:
    - Treino: 100%
    - Validação: 80.2%
    
Ao interpretar estes dados, concluimos que o SVM desempenha uma melhor classificação se o treinarmos com um dicionário pré-processado com o Snowball Stemmer.<br>
Outra clara conclusão já retirada previamente, os vocabulários a incluir bi-gramas e tri-gramas são muito maiores que apenas com unigramas e bi-gramas, como era expectável. Apresentando uma lista de tamanhos para uma comparação facilitada:  

- No Stem:
    - Unigramas: 5666
    - Unigramas e Bi-gramas: 30561
    - Unigramas, Bi-gramas e Tri-gramas: 46556
- Porter Stemmer:
    - Unigramas: 4326
    - Unigramas e Bi-gramas: 29675
    - Unigramas, Bi-gramas e Tri-gramas: 47054
- Snowball Stemmer:
    - Unigramas: 4256
    - Unigramas e Bi-gramas: 29622
    - Unigramas, Bi-gramas e Tri-gramas: 47066
- Lancaster Stemmer:
    - Unigramas: 3391
    - Unigramas e Bi-gramas: 29316
    - Unigramas, Bi-gramas e Tri-gramas: 47129

Novamente, é possível observar o aumento exponencial de um caso para outro e, consequentemente, traduz-se no aumento considerado de tempo de treino e avaliação dos classificadores.

Após estas considerações, antes de avançar para uma iteração final, vamos fazer alguns testes para um valor de gama superior a 1, para vocabulários sem stemming e com cada tipo de stemming, mas apenas com unigramas.

In [137]:
tfidf_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a)
X1_noStem = tfidf_noStem.transform(Docs1)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
X2_noStem = tfidf_noStem.transform(Docs2)
tokens_noStem = tfidf_noStem.get_feature_names()

tfidf_porter = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a_PorterStem)
X1_porter = tfidf_porter.transform(Docs1_PorterStem)
X1a_porter = tfidf_porter.transform(Docs1a_PorterStem)
X1b_porter = tfidf_porter.transform(Docs1b_PorterStem)
X2_porter = tfidf_porter.transform(Docs2_PorterStem)
tokens_porter = tfidf_porter.get_feature_names()

tfidf_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_snowball = tfidf_snowball.transform(Docs1a_SnowballStem)
X1b_snowball = tfidf_snowball.transform(Docs1b_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

tfidf_lancaster = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1a_LancasterStem)
X1_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_lancaster = tfidf_lancaster.transform(Docs1a_LancasterStem)
X1b_lancaster = tfidf_lancaster.transform(Docs1b_LancasterStem)
X2_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_lancaster = tfidf_lancaster.get_feature_names()

In [138]:
print(len(tokens_noStem))
print(len(tokens_porter))
print(len(tokens_snowball))
print(len(tokens_lancaster))

6175
4885
4813
4292


### SVM - No Stem

In [139]:
topScore = 0
ListG = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  80 . G =  1.5  | C =  0.1  | Score: 0.6114285714285714  | Time:  0:35.06518316268921
2  -  80 . G =  2  | C =  0.1  | Score: 0.5507142857142857  | Time:  0:32.9495153427124
3  -  80 . G =  2.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:36.354222774505615
4  -  80 . G =  3  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:40.367878913879395
5  -  80 . G =  3.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.46207046508789
6  -  80 . G =  4  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:34.1776704788208
7  -  80 . G =  4.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.397491216659546
8  -  80 . G =  5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.38090682029724
9  -  80 . G =  1.5  | C =  0.5  | Score: 0.7946428571428571  | Time:  0:29.64991068840027
10  -  80 . G =  2  | C =  0.5  | Score: 0.7771428571428571  | Time:  0:32.23714637756348
11  -  80 . G =  2.5  | C =  0.5  | Score: 0.7175  | Time:  0:33.28814387321472
12  -  80 . G =  3

### SVM - Porter Stemmer

In [142]:
begin = time.time()

topScore = 0
ListG = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

0  -  80 . G =  1.5  | C =  0.1  | Score: 0.6492857142857142  | Time:  0:33.34936881065369
1  -  80 . G =  2  | C =  0.1  | Score: 0.5589285714285714  | Time:  0:32.540964126586914
2  -  80 . G =  2.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.610410928726196
3  -  80 . G =  3  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.91412568092346
4  -  80 . G =  3.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:34.57350564002991
5  -  80 . G =  4  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:34.059550285339355
6  -  80 . G =  4.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.89224195480347
7  -  80 . G =  5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.92583680152893
8  -  80 . G =  1.5  | C =  0.5  | Score: 0.7985714285714286  | Time:  0:28.369069814682007
9  -  80 . G =  2  | C =  0.5  | Score: 0.79  | Time:  0:30.856022357940674
10  -  80 . G =  2.5  | C =  0.5  | Score: 0.7435714285714285  | Time:  0:32.65151786804199
11  -  80 . G =  

### SVM - Snowball Stemmer

In [144]:
begin = time.time()

topScore = 0
ListG = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  80 . G =  1.5  | C =  0.1  | Score: 0.6510714285714285  | Time:  0:31.204721450805664
2  -  80 . G =  2  | C =  0.1  | Score: 0.5592857142857143  | Time:  0:31.626726388931274
3  -  80 . G =  2.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:32.35161805152893
4  -  80 . G =  3  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.13552236557007
5  -  80 . G =  3.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.016035318374634
6  -  80 . G =  4  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:33.15836453437805
7  -  80 . G =  4.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:32.93196129798889
8  -  80 . G =  5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:32.919251441955566
9  -  80 . G =  1.5  | C =  0.5  | Score: 0.8003571428571429  | Time:  0:27.998148441314697
10  -  80 . G =  2  | C =  0.5  | Score: 0.79  | Time:  0:30.546750783920288
11  -  80 . G =  2.5  | C =  0.5  | Score: 0.7442857142857143  | Time:  0:32.24536085128784
12  -  80 . G =

### SVM - Lancaster Stemmer

In [146]:
begin = time.time()

topScore = 0
ListG = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

1  -  80 . G =  1.5  | C =  0.1  | Score: 0.6557142857142857  | Time:  0:33.331793546676636
2  -  80 . G =  2  | C =  0.1  | Score: 0.5642857142857143  | Time:  0:35.00789952278137
3  -  80 . G =  2.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:36.77452230453491
4  -  80 . G =  3  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:38.333309173583984
5  -  80 . G =  3.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:36.51159644126892
6  -  80 . G =  4  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:37.5239098072052
7  -  80 . G =  4.5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:37.95330333709717
8  -  80 . G =  5  | C =  0.1  | Score: 0.5503571428571429  | Time:  0:38.62189984321594
9  -  80 . G =  1.5  | C =  0.5  | Score: 0.8  | Time:  0:32.44086241722107
10  -  80 . G =  2  | C =  0.5  | Score: 0.7882142857142858  | Time:  0:34.952146768569946
11  -  80 . G =  2.5  | C =  0.5  | Score: 0.7475  | Time:  0:39.1316180229187
12  -  80 . G =  3  | C =  0.5  |

Conclusões tiradas nas classificações com SVM:
- Sem stemming:
    - Treino:  98.7%
    - Validação: 80.2%
- Porter Stemming:
    - Treino: 100%
    - Validação: 80.4%
- Snowball Stemming:
    - Treino: 100%
    - Validação: 80.4%
- Lancaster Stemming:
    - Treino: 100%
    - Validação: 80.1%
    
Ao interpretar estes dados, concluimos que o SVM desempenha uma melhor classificação se o treinarmos com um dicionário pré-processado com o Porter Stemmer ou o Snowball Stemmer.<br>
Outra clara conclusão já retirada previamente, os vocabulários a incluir bi-gramas e tri-gramas são muito maiores que apenas com unigramas e bi-gramas, como era expectável. Apresentando uma lista de tamanhos para uma comparação facilitada:  

É possível concluir que valores de gama acima de 1.5 obtêm maus resultados.

Com todos estas variedades de teste, podemos verificar que não existem grandes diferenças de resultados caso o vocabulário possua dados pré-processados com stemming ou sem.<br>
Outra conclusão é o melhor desempenho na classificação dos dados com o uso de SVM's em vez do uso do LogisticRegression. Embora a diferença não seja muito acentuada.

Assim, com base em todas as variedades de pré-processamento de dados e parametros dos SVM's e LogisticRegression, os dados têm de ser pré-processados com o Snowball Stemmer e os hiper-parametro do TfidfVectorizer, para um melhor resultado, são "min_df" igual a "3", "token-pattern" igual a "r'\b\w\w+\b" e "ngram_range" igual a "(1, 3)". Usaremos uma Máquina de Suporte Vectorial com os hiper-parametros "C" igual a "1" e gama igual a "0.25".

### Terceira Iteração

In [150]:
snowballStemFunc = SnowballStemmer(language='english')
Docs1_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs2_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs2]

tfidf_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b', ngram_range=(1, 3)).fit(Docs1_SnowballStem)
X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_snowball = tfidf_snowball.get_feature_names()

#begin = time.time()

svm_snowball = SVC(kernel = 'rbf', gamma = 0.25, C = 1).fit(X1_snowball, y_Train_ClassBin)
         
#end = time.time()

# timeTaken = end - begin
# minutes = np.floor(timeTaken / 60)
# seconds = timeTaken % 60
# timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

#print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1_snowball, y_Train_ClassBin))
print("% de acertos no teste: ", svm_snowball.score(X2_snowball, y_Test_ClassBin))
#print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

% de acertos no treino:  0.8967611336032388
% de acertos no teste:  0.8813559322033898


In [83]:
# Posso testar isto mais tarde para ver se existem diferenças dos dados estarem, ou não, normalizados
# sc = StandardScaler().fit(X1)
# X1n = sc.transform(X1)
# X2n = sc.transform(X2)

## NOTA PARA CLASSIFICAÇÃO BINÁRIA:
- A intenção é ter dois classificadores diferentes. Passamos as criticas TODAS de treino, com todo o tipo de classificações.<br>
Classificadores:
    - Muito boas, 2 classes: negativo - Tudo abaixo de 9; positivo - 9 e 10.
    - Muito más, 2 classes: negativo - Tudo acima de 2; positivo - 1 e 2.

In [144]:
import pickle
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from sklearn.svm import SVC
import time

In [145]:
D_train = pickle.load(open('rateBeer75Ktrain.p', 'rb'))
D_test = pickle.load(open('rateBeer25Ktest.p', 'rb'))

In [146]:
#Arranjar os X's do treino.
X1 = np.array([valor["review"] for valor in D_train.values()])

#Arranjar os y's do treino.
y1Feel = np.array([int(valor["feel"][0]) for valor in D_train.values()])
y1Look = np.array([int(valor["look"][0]) for valor in D_train.values()])
y1Smell = np.array([int(valor["smell"][0]) for valor in D_train.values()])
y1Taste = np.array([int(valor["taste"][0]) for valor in D_train.values()])
y1Overall = np.array([int(valor["overall"][0]) for valor in D_train.values()])

#Arranjar os X's de teste.
X2 = np.array([valor["review"] for valor in D_test.values()])

#Arranjar os y's do teste.
y2Feel = np.array([int(valor["feel"][0]) for valor in D_test.values()])
y2Look = np.array([int(valor["look"][0]) for valor in D_test.values()])
y2Smell = np.array([int(valor["smell"][0]) for valor in D_test.values()])
y2Taste = np.array([int(valor["taste"][0]) for valor in D_test.values()])
y2Overall = np.array([int(valor["overall"][0]) for valor in D_test.values()])

In [147]:
y1BinClass_MuitoBoas = np.where(y1Overall < 9, -1, 1)
y1BinClass_MuitoMas = np.where(y1Overall > 2, -1, 1)

y2BinClass_MuitoBoas = np.where(y2Overall < 9, -1, 1)
y2BinClass_MuitoMas = np.where(y2Overall > 2, -1, 1)

In [148]:
#Divisão em Dados de Treino e Dados de Validação
X1aBinClass_MuitoBoas, X1bBinClass_MuitoBoas, y1aBinClass_MuitoBoas, y1bBinClass_MuitoBoas = train_test_split(X1, y1BinClass_MuitoBoas, test_size=1/3, stratify=y1BinClass_MuitoBoas, random_state=42)
X1aBinClass_MuitoMas, X1bBinClass_MuitoMas, y1aBinClass_MuitoMas, y1bBinClass_MuitoMas = train_test_split(X1, y1BinClass_MuitoMas, test_size=1/3, stratify=y1BinClass_MuitoMas, random_state=42)

In [149]:
#Criação do Dicionário
Docs1 = [doc.replace('<br />', ' ') for doc in X1]
Docs1 = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1]

Docs2 = [doc.replace('<br />', ' ') for doc in X2]
Docs2 = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs2]


#Especificação Criticas Muito Boas
Docs1aMuitoBoas = [doc.replace('<br />', ' ') for doc in X1aBinClass_MuitoBoas]
Docs1aMuitoBoas = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1aMuitoBoas]

Docs1bMuitoBoas = [doc.replace('<br />', ' ') for doc in X1bBinClass_MuitoBoas]
Docs1bMuitoBoas = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1bMuitoBoas]


#Especificação Criticas Muito Mas
Docs1aMuitoMas = [doc.replace('<br />', ' ') for doc in X1aBinClass_MuitoMas]
Docs1aMuitoMas = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1aMuitoMas]

Docs1bMuitoMas = [doc.replace('<br />', ' ') for doc in X1bBinClass_MuitoMas]
Docs1bMuitoMas = [re.sub(r'[^a-zA-Z\u00C0-\u00FF]+', ' ', doc) for doc in Docs1bMuitoMas]


#Stemming
porterStemFunc = PorterStemmer()
Docs1_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs2_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs2]

snowballStemFunc = SnowballStemmer(language='english')
Docs1_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs2_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs2]

lancasterStemFunc = LancasterStemmer()
Docs1_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1]
Docs2_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs2]


#Especificação Criticas Muito Boas
Docs1aMuitoBoas_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1aMuitoBoas]
Docs1bMuitoBoas_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1bMuitoBoas]

Docs1aMuitoBoas_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1aMuitoBoas]
Docs1bMuitoBoas_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1bMuitoBoas]

Docs1aMuitoBoas_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1aMuitoBoas]
Docs1bMuitoBoas_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1bMuitoBoas]


#Especificação Criticas Muito Mas
Docs1aMuitoMas_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1aMuitoMas]
Docs1bMuitoMas_PorterStem =[' '.join([porterStemFunc.stem(w) for w in d.split()]) for d in Docs1bMuitoMas]

Docs1aMuitoMas_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1aMuitoMas]
Docs1bMuitoMas_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1bMuitoMas]

Docs1aMuitoMas_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1aMuitoMas]
Docs1bMuitoMas_LancasterStem =[' '.join([lancasterStemFunc.stem(w) for w in d.split()]) for d in Docs1bMuitoMas]

In [150]:
#Vectorização das Criticas - No Stem - Muito Boas
tfidf_MuitoBoas_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoBoas)
X1_MuitoBoas_noStem = tfidf_MuitoBoas_noStem.transform(Docs1)
X1a_MuitoBoas_noStem = tfidf_MuitoBoas_noStem.transform(Docs1aMuitoBoas)
X1b_MuitoBoas_noStem = tfidf_MuitoBoas_noStem.transform(Docs1bMuitoBoas)
X2_MuitoBoas_noStem = tfidf_MuitoBoas_noStem.transform(Docs2)
tokens_MuitoBoas_noStem = tfidf_MuitoBoas_noStem.get_feature_names()


#Vectorização das Criticas - Porter Stemming - Muito Boas
tfidf_MuitoBoas_porter = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoBoas_PorterStem)
X1_MuitoBoas_porter = tfidf_MuitoBoas_porter.transform(Docs1_PorterStem)
X1a_MuitoBoas_porter = tfidf_MuitoBoas_porter.transform(Docs1aMuitoBoas_PorterStem)
X1b_MuitoBoas_porter = tfidf_MuitoBoas_porter.transform(Docs1bMuitoBoas_PorterStem)
X2_MuitoBoas_porter = tfidf_MuitoBoas_porter.transform(Docs2_PorterStem)
tokens_MuitoBoas_porter = tfidf_MuitoBoas_porter.get_feature_names()


#Vectorização das Criticas - Snowball Stemming - Muito Boas
tfidf_MuitoBoas_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoBoas_SnowballStem)
X1_MuitoBoas_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
X1a_MuitoBoas_snowball = tfidf_snowball.transform(Docs1aMuitoBoas_SnowballStem)
X1b_MuitoBoas_snowball = tfidf_snowball.transform(Docs1bMuitoBoas_SnowballStem)
X2_MuitoBoas_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
tokens_MuitoBoas_snowball = tfidf_snowball.get_feature_names()


#Vectorização das Criticas - Lancaster Stemming - Muito Boas
tfidf_lancaster = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoBoas_LancasterStem)
X1_MuitoBoas_lancaster = tfidf_lancaster.transform(Docs1_LancasterStem)
X1a_MuitoBoas_lancaster = tfidf_lancaster.transform(Docs1aMuitoBoas_LancasterStem)
X1b_MuitoBoas_lancaster = tfidf_lancaster.transform(Docs1bMuitoBoas_LancasterStem)
X2_MuitoBoas_lancaster = tfidf_lancaster.transform(Docs2_LancasterStem)
tokens_MuitoBoas_lancaster = tfidf_lancaster.get_feature_names()

#------------------------------------#
#------------------------------------#

#Vectorização das Criticas - No Stem - Muito Mas
tfidf_MuitoMas_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoMas)
X1_MuitoMas_noStem = tfidf_MuitoMas_noStem.transform(Docs1)
X1a_MuitoMas_noStem = tfidf_MuitoMas_noStem.transform(Docs1aMuitoMas)
X1b_MuitoMas_noStem = tfidf_MuitoMas_noStem.transform(Docs1bMuitoMas)
X2_MuitoMas_noStem = tfidf_MuitoMas_noStem.transform(Docs2)
tokens_MuitoMas_noStem = tfidf_MuitoMas_noStem.get_feature_names()


#Vectorização das Criticas - Porter Stemming - Muito Mas
tfidf_MuitoMas_porter = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoMas_PorterStem)
X1_MuitoMas_porter = tfidf_MuitoMas_porter.transform(Docs1_PorterStem)
X1a_MuitoMas_porter = tfidf_MuitoMas_porter.transform(Docs1aMuitoMas_PorterStem)
X1b_MuitoMas_porter = tfidf_MuitoMas_porter.transform(Docs1bMuitoMas_PorterStem)
X2_MuitoMas_porter = tfidf_MuitoMas_porter.transform(Docs2_PorterStem)
tokens_MuitoMas_porter = tfidf_MuitoMas_porter.get_feature_names()


#Vectorização das Criticas - Snowball Stemming - Muito Mas
tfidf_MuitoMas_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoMas_SnowballStem)
X1_MuitoMas_snowball = tfidf_MuitoMas_snowball.transform(Docs1_SnowballStem)
X1a_MuitoMas_snowball = tfidf_MuitoMas_snowball.transform(Docs1aMuitoMas_SnowballStem)
X1b_MuitoMas_snowball = tfidf_MuitoMas_snowball.transform(Docs1bMuitoMas_SnowballStem)
X2_MuitoMas_snowball = tfidf_MuitoMas_snowball.transform(Docs2_SnowballStem)
tokens_MuitoMas_snowball = tfidf_MuitoMas_snowball.get_feature_names()


#Vectorização das Criticas - Lancaster Stemming - Muito Mas
tfidf_MuitoMas_lancaster = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b').fit(Docs1aMuitoMas_LancasterStem)
X1_MuitoMas_lancaster = tfidf_MuitoMas_lancaster.transform(Docs1_LancasterStem)
X1a_MuitoMas_lancaster = tfidf_MuitoMas_lancaster.transform(Docs1aMuitoMas_LancasterStem)
X1b_MuitoMas_lancaster = tfidf_MuitoMas_lancaster.transform(Docs1bMuitoMas_LancasterStem)
X2_MuitoMas_lancaster = tfidf_MuitoMas_lancaster.transform(Docs2_LancasterStem)
tokens_MuitoMas_lancaster = tfidf_MuitoMas_lancaster.get_feature_names()

#### Classificação Binária: Criticas Muito Boas

##### No Stem

In [36]:
#Forçar hiperparâmetros - Muito Boas
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScoreLasso = 0
topScoreRidge = 0
index = 1
maxIndex = len(ListC)

begin = time.time()

for c in ListC:
    timeBeginClass = time.time()

    #Lasso
    beginLasso = time.time()
    dlMuitoBoas_NoStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_NoStem_Lasso.fit(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas)
    scoreLasso = dlMuitoBoas_NoStem_Lasso.score(X1b_MuitoBoas_noStem, y1bBinClass_MuitoBoas)
    endLasso = time.time()

    #Time - Lasso
    timeTakenLasso = endLasso - beginLasso
    minutesLasso = np.floor(timeTakenLasso / 60)
    secondsLasso = timeTakenLasso % 60
    timeTakenLassoDisplay = str(int(minutesLasso)) + ":" + str(int(secondsLasso))
   
    #Ridge
    beginRidge = time.time()
    dlMuitoBoas_NoStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_NoStem_Ridge.fit(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas)
    scoreRidge = dlMuitoBoas_NoStem_Ridge.score(X1b_MuitoBoas_noStem, y1bBinClass_MuitoBoas)
    endRidge = time.time()
    
    #Time - Ridge
    timeTakenRidge = endRidge - beginRidge
    minutesRidge = np.floor(timeTakenRidge / 60)
    secondsRidge = timeTakenRidge % 60
    timeTakenRidgeDisplay = str(int(minutesRidge)) + ":" + str(int(secondsRidge))

    #Time - Total
    timeTakenClass = time.time() - timeBeginClass
    minutes = np.floor(timeTakenClass / 60)
    seconds = timeTakenClass % 60
    timeTakenDisplay = str(int(minutes)) + ":" + str(int(seconds))

    if scoreLasso >= topScoreLasso:
        topScoreLasso = scoreLasso
        bestCLasso = c
    
    if scoreRidge >= topScoreRidge:
        topScoreRidge = scoreRidge
        bestCRidge = c

    print(index, " - ", maxIndex, ". C = ", c)
    print("Score Lasso: ", scoreLasso, " | Time Lasso: ", timeTakenLassoDisplay)
    print("Score Ridge: ", scoreRidge, " | Time Ridge: ", timeTakenRidgeDisplay)
    print("Time Taken Index: ", timeTakenDisplay)
    print("-----------------------------------------------------")
    
    index = index + 1

    
dlMuitoBoas_NoStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = bestCLasso, tol = 1e-3)
dlMuitoBoas_NoStem_Lasso.fit(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas)

dlMuitoBoas_NoStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = bestCRidge, tol = 1e-3)
dlMuitoBoas_NoStem_Ridge.fit(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print()
print("---- Classificação Binária - Cervejas Muito Boas ----")
print("Regularização Lasso")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_NoStem_Lasso.score(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_NoStem_Lasso.score(X1b_MuitoBoas_noStem, y1bBinClass_MuitoBoas) * 100)
print("Tempo Demorado: ", timeTakenLassoDisplay)
print("--------------------------")
print("Regularização Ridge")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_NoStem_Ridge.score(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_NoStem_Ridge.score(X1b_MuitoBoas_noStem, y1bBinClass_MuitoBoas) * 100)
print("Tempo Demorado: ", timeTakenRidgeDisplay)
print("--------------------------")
print("Total Tempo Demorado: ", timeTakenTotalDisplay)

# print()
# print("---- Classificação Binária - Cervejas Muito Más ----")
# print("% Acertos nos Dados de Treino: ", dlMuitoMas.score(X_Train_tfidf, y_Train_ClassBin_MuitoMas) * 100)
# print("% Acertos nos Dados de Teste: ", dlMuitoMas.score(X_Test_tfidf, y_Test_ClassBin_MuitoMas) * 100)

1  -  10 . C =  0.1
Score Lasso:  0.94968  | Time Lasso:  0:2
Score Ridge:  0.94968  | Time Ridge:  0:0
Time Taken Index:  0:3
-----------------------------------------------------
2  -  10 . C =  0.5
Score Lasso:  0.94952  | Time Lasso:  0:3
Score Ridge:  0.94952  | Time Ridge:  0:0
Time Taken Index:  0:4
-----------------------------------------------------
3  -  10 . C =  1
Score Lasso:  0.94912  | Time Lasso:  0:10
Score Ridge:  0.94912  | Time Ridge:  0:1
Time Taken Index:  0:11
-----------------------------------------------------
4  -  10 . C =  2.5
Score Lasso:  0.94664  | Time Lasso:  1:7
Score Ridge:  0.94832  | Time Ridge:  0:2
Time Taken Index:  1:9
-----------------------------------------------------
5  -  10 . C =  5
Score Lasso:  0.94264  | Time Lasso:  2:28
Score Ridge:  0.94748  | Time Ridge:  0:2
Time Taken Index:  2:31
-----------------------------------------------------
6  -  10 . C =  10
Score Lasso:  0.93692  | Time Lasso:  5:49
Score Ridge:  0.9458  | Time Ridg

##### Porter Stemmer

In [52]:
#Forçar hiperparâmetros - Muito Boas
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100]
topScoreNoPenalty = 0
topScoreLasso = 0
topScoreRidge = 0
index = 1
maxIndex = len(ListC)

begin = time.time()

for c in ListC:
    timeBeginClass = time.time()

#     #Sem Penalização
#     beginLasso = time.time()
#     dlMuitoBoas_PorterStem_NoPenalty = LogisticRegression(max_iter = 1000, C = c, tol = 1e-3)
#     dlMuitoBoas_PorterStem_NoPenalty.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
#     scoreNoPenalty = dlMuitoBoas_PorterStem_NoPenalty.score(X1b_MuitoBoas_porter, y1bBinClass_MuitoBoas)
#     endNoPenalty = time.time()

#     #Time - Sem Penalização
#     timeTakenNoPenalty = endNoPenalty - beginNoPenalty
#     minutesNoPenalty = np.floor(timeTakenNoPenalty / 60)
#     secondsNoPenalty = timeTakenNoPenalty % 60
#     timeTakenNoPenaltyDisplay = str(int(minutesNoPenalty)) + ":" + str(int(secondsNoPenalty))

    
    #Lasso
    beginLasso = time.time()
    dlMuitoBoas_PorterStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_PorterStem_Lasso.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
    scoreLasso = dlMuitoBoas_PorterStem_Lasso.score(X1b_MuitoBoas_porter, y1bBinClass_MuitoBoas)
    endLasso = time.time()

    #Time - Lasso
    timeTakenLasso = endLasso - beginLasso
    minutesLasso = np.floor(timeTakenLasso / 60)
    secondsLasso = timeTakenLasso % 60
    timeTakenLassoDisplay = str(int(minutesLasso)) + ":" + str(int(secondsLasso))
   


    #Ridge
    beginRidge = time.time()
    dlMuitoBoas_PorterStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_PorterStem_Ridge.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
    scoreRidge = dlMuitoBoas_PorterStem_Ridge.score(X1b_MuitoBoas_porter, y1bBinClass_MuitoBoas)
    endRidge = time.time()
    
    #Time - Ridge
    timeTakenRidge = endRidge - beginRidge
    minutesRidge = np.floor(timeTakenRidge / 60)
    secondsRidge = timeTakenRidge % 60
    timeTakenRidgeDisplay = str(int(minutesRidge)) + ":" + str(int(secondsRidge))

    #Time - Total
    timeTakenClass = time.time() - timeBeginClass
    minutes = np.floor(timeTakenClass / 60)
    seconds = timeTakenClass % 60
    timeTakenDisplay = str(int(minutes)) + ":" + str(int(seconds))

#     if scoreNoPenalty >= topScoreNoPenalty:
#         topScoreNoPenalty = scoreNoPenalty
#         bestCNoPenalty = c
    
    if scoreLasso >= topScoreLasso:
        topScoreLasso = scoreLasso
        bestCLasso = c
    
    if scoreRidge >= topScoreRidge:
        topScoreRidge = scoreRidge
        bestCRidge = c

    print(index, " - ", maxIndex, ". C = ", c)
#     print("Score No Penalty: ", scoreNoPenalty, " | Time No Penalty: ", timeTakenNoPenaltyDisplay)
    print("Score Lasso: ", scoreLasso, " | Time Lasso: ", timeTakenLassoDisplay)
    print("Score Ridge: ", scoreRidge, " | Time Ridge: ", timeTakenRidgeDisplay)
    print("Time Taken Index: ", timeTakenDisplay)
    print("-----------------------------------------------------")
    
    index = index + 1

# beginNoPenalty = time.time()
# dlMuitoBoas_PorterStem_NoPenalty = LogisticRegression(max_iter = 1000, C = bestCNoPenalty, tol = 1e-3)
# dlMuitoBoas_PorterStem_NoPenalty.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
# endNoPenalty = time.time()
   
beginLasso = time.time()
dlMuitoBoas_PorterStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = bestCLasso, tol = 1e-3)
dlMuitoBoas_PorterStem_Lasso.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
endLasso = time.time()

beginRidge = time.time()
dlMuitoBoas_PorterStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = bestCRidge, tol = 1e-3)
dlMuitoBoas_PorterStem_Ridge.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
endRidge = time.time()

end = time.time()

# timeTakenNoPenalty = endNoPenalty - beginNoPenalty
# minutesNoPenalty = np.floor(timeTakenNoPenalty / 60)
# secondsNoPenalty = timeTakenNoPenalty % 60
# timeTakenNoPenaltyDisplay = str(int(minutesNoPenalty)) + ":" + str(int(secondsNoPenalty))
    
timeTakenLasso = endLasso - beginLasso
minutesLasso = np.floor(timeTakenLasso / 60)
secondsLasso = timeTakenLasso % 60
timeTakenLassoDisplay = str(int(minutesLasso)) + ":" + str(int(secondsLasso))
    
timeTakenRidge = endRidge - beginRidge
minutesRidge = np.floor(timeTakenRidge / 60)
secondsRidge = timeTakenRidge % 60
timeTakenRidgeDisplay = str(int(minutesRidge)) + ":" + str(int(secondsRidge))

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print()
print("---- Classificação Binária - Cervejas Muito Boas ----")
# print("Sem Regularização")
# print("% Acertos nos Dados de Treino: ", dlMuitoBoas_PorterStem_NoPenalty.score(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas) * 100)
# print("% Acertos nos Dados de Validação: ", dlMuitoBoas_PorterStem_NoPenalty.score(X1b_MuitoBoas_porter, y1bBinClass_MuitoBoas) * 100)
# print("Melhor C: ", bestCNoPenalty)
# print("Tempo Demorado: ", timeTakenNoPenaltyDisplay)
# print("--------------------------")
print("Regularização Lasso")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_PorterStem_Lasso.score(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_PorterStem_Lasso.score(X1b_MuitoBoas_porter, y1bBinClass_MuitoBoas) * 100)
print("Melhor C: ", bestCLasso)
print("Tempo Demorado: ", timeTakenLassoDisplay)
print("--------------------------")
print("Regularização Ridge")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_PorterStem_Ridge.score(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_PorterStem_Ridge.score(X1b_MuitoBoas_porter, y1bBinClass_MuitoBoas) * 100)
print("Melhor C: ", bestCRidge)
print("Tempo Demorado: ", timeTakenRidgeDisplay)
print("--------------------------")
print("Total Tempo Demorado: ", timeTakenTotalDisplay)

# print()
# print("---- Classificação Binária - Cervejas Muito Más ----")
# print("% Acertos nos Dados de Treino: ", dlMuitoMas.score(X_Train_tfidf, y_Train_ClassBin_MuitoMas) * 100)
# print("% Acertos nos Dados de Teste: ", dlMuitoMas.score(X_Test_tfidf, y_Test_ClassBin_MuitoMas) * 100)

1  -  9 . C =  0.1
Score Lasso:  0.94972  | Time Lasso:  0:1
Score Ridge:  0.94968  | Time Ridge:  0:0
Time Taken Index:  0:2
-----------------------------------------------------
2  -  9 . C =  0.5
Score Lasso:  0.94932  | Time Lasso:  0:3
Score Ridge:  0.94948  | Time Ridge:  0:0
Time Taken Index:  0:3
-----------------------------------------------------
3  -  9 . C =  1
Score Lasso:  0.9492  | Time Lasso:  0:9
Score Ridge:  0.94948  | Time Ridge:  0:1
Time Taken Index:  0:11
-----------------------------------------------------
4  -  9 . C =  2.5
Score Lasso:  0.94672  | Time Lasso:  0:46
Score Ridge:  0.94828  | Time Ridge:  0:1
Time Taken Index:  0:48
-----------------------------------------------------
5  -  9 . C =  5
Score Lasso:  0.94372  | Time Lasso:  2:25
Score Ridge:  0.94736  | Time Ridge:  0:3
Time Taken Index:  2:28
-----------------------------------------------------
6  -  9 . C =  10
Score Lasso:  0.93644  | Time Lasso:  5:1
Score Ridge:  0.9456  | Time Ridge:  0:3

##### Snowball Stemmer

In [53]:
#Forçar hiperparâmetros - Muito Boas
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100]
topScoreLasso = 0
topScoreRidge = 0
index = 1
maxIndex = len(ListC)

begin = time.time()

for c in ListC:
    timeBeginClass = time.time()

    #Lasso
    beginLasso = time.time()
    dlMuitoBoas_SnowballStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_SnowballStem_Lasso.fit(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas)
    scoreLasso = dlMuitoBoas_SnowballStem_Lasso.score(X1b_MuitoBoas_snowball, y1bBinClass_MuitoBoas)
    endLasso = time.time()

    #Time - Lasso
    timeTakenLasso = endLasso - beginLasso
    minutesLasso = np.floor(timeTakenLasso / 60)
    secondsLasso = timeTakenLasso % 60
    timeTakenLassoDisplay = str(int(minutesLasso)) + ":" + str(int(secondsLasso))
   
    #Ridge
    beginRidge = time.time()
    dlMuitoBoas_SnowballStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_SnowballStem_Ridge.fit(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas)
    scoreRidge = dlMuitoBoas_SnowballStem_Ridge.score(X1b_MuitoBoas_snowball, y1bBinClass_MuitoBoas)
    endRidge = time.time()
    
    #Time - Ridge
    timeTakenRidge = endRidge - beginRidge
    minutesRidge = np.floor(timeTakenRidge / 60)
    secondsRidge = timeTakenRidge % 60
    timeTakenRidgeDisplay = str(int(minutesRidge)) + ":" + str(int(secondsRidge))

    #Time - Total
    timeTakenClass = time.time() - timeBeginClass
    minutes = np.floor(timeTakenClass / 60)
    seconds = timeTakenClass % 60
    timeTakenDisplay = str(int(minutes)) + ":" + str(int(seconds))

    if scoreLasso >= topScoreLasso:
        topScoreLasso = scoreLasso
        bestCLasso = c
    
    if scoreRidge >= topScoreRidge:
        topScoreRidge = scoreRidge
        bestCRidge = c

    print(index, " - ", maxIndex, ". C = ", c)
    print("Score Lasso: ", scoreLasso, " | Time Lasso: ", timeTakenLassoDisplay)
    print("Score Ridge: ", scoreRidge, " | Time Ridge: ", timeTakenRidgeDisplay)
    print("Time Taken Index: ", timeTakenDisplay)
    print("-----------------------------------------------------")
    
    index = index + 1

    
dlMuitoBoas_SnowballStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = bestCLasso, tol = 1e-3)
dlMuitoBoas_SnowballStem_Lasso.fit(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas)

dlMuitoBoas_SnowballStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = bestCRidge, tol = 1e-3)
dlMuitoBoas_SnowballStem_Ridge.fit(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print()
print("---- Classificação Binária - Cervejas Muito Boas ----")
print("Regularização Lasso")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_SnowballStem_Lasso.score(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_SnowballStem_Lasso.score(X1b_MuitoBoas_snowball, y1bBinClass_MuitoBoas) * 100)
print("Melhor C: ", bestCLasso)
print("Tempo Demorado: ", timeTakenLassoDisplay)
print("--------------------------")
print("Regularização Ridge")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_SnowballStem_Ridge.score(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_SnowballStem_Ridge.score(X1b_MuitoBoas_snowball, y1bBinClass_MuitoBoas) * 100)
print("Melhor C: ", bestCRidge)
print("Tempo Demorado: ", timeTakenRidgeDisplay)
print("--------------------------")
print("Total Tempo Demorado: ", timeTakenTotalDisplay)

# print()
# print("---- Classificação Binária - Cervejas Muito Más ----")
# print("% Acertos nos Dados de Treino: ", dlMuitoMas.score(X_Train_tfidf, y_Train_ClassBin_MuitoMas) * 100)
# print("% Acertos nos Dados de Teste: ", dlMuitoMas.score(X_Test_tfidf, y_Test_ClassBin_MuitoMas) * 100)

1  -  9 . C =  0.1
Score Lasso:  0.94972  | Time Lasso:  0:2
Score Ridge:  0.94968  | Time Ridge:  0:1
Time Taken Index:  0:3
-----------------------------------------------------
2  -  9 . C =  0.5
Score Lasso:  0.94932  | Time Lasso:  0:2
Score Ridge:  0.94964  | Time Ridge:  0:0
Time Taken Index:  0:3
-----------------------------------------------------
3  -  9 . C =  1
Score Lasso:  0.9492  | Time Lasso:  0:8
Score Ridge:  0.94948  | Time Ridge:  0:1
Time Taken Index:  0:9
-----------------------------------------------------
4  -  9 . C =  2.5
Score Lasso:  0.94724  | Time Lasso:  0:37
Score Ridge:  0.94824  | Time Ridge:  0:1
Time Taken Index:  0:39
-----------------------------------------------------
5  -  9 . C =  5
Score Lasso:  0.94316  | Time Lasso:  2:36
Score Ridge:  0.94764  | Time Ridge:  0:3
Time Taken Index:  2:40
-----------------------------------------------------
6  -  9 . C =  10
Score Lasso:  0.93572  | Time Lasso:  5:55
Score Ridge:  0.9458  | Time Ridge:  0:4

##### Lancaster Stemmer

In [54]:
#Forçar hiperparâmetros - Muito Boas
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100]
topScoreLasso = 0
topScoreRidge = 0
index = 1
maxIndex = len(ListC)

begin = time.time()

for c in ListC:
    timeBeginClass = time.time()

    #Lasso
    beginLasso = time.time()
    dlMuitoBoas_LancasterStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_LancasterStem_Lasso.fit(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas)
    scoreLasso = dlMuitoBoas_LancasterStem_Lasso.score(X1b_MuitoBoas_lancaster, y1bBinClass_MuitoBoas)
    endLasso = time.time()

    #Time - Lasso
    timeTakenLasso = endLasso - beginLasso
    minutesLasso = np.floor(timeTakenLasso / 60)
    secondsLasso = timeTakenLasso % 60
    timeTakenLassoDisplay = str(int(minutesLasso)) + ":" + str(int(secondsLasso))
   
    #Ridge
    beginRidge = time.time()
    dlMuitoBoas_LancasterStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoBoas_LancasterStem_Ridge.fit(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas)
    scoreRidge = dlMuitoBoas_LancasterStem_Ridge.score(X1b_MuitoBoas_lancaster, y1bBinClass_MuitoBoas)
    endRidge = time.time()
    
    #Time - Ridge
    timeTakenRidge = endRidge - beginRidge
    minutesRidge = np.floor(timeTakenRidge / 60)
    secondsRidge = timeTakenRidge % 60
    timeTakenRidgeDisplay = str(int(minutesRidge)) + ":" + str(int(secondsRidge))

    #Time - Total
    timeTakenClass = time.time() - timeBeginClass
    minutes = np.floor(timeTakenClass / 60)
    seconds = timeTakenClass % 60
    timeTakenDisplay = str(int(minutes)) + ":" + str(int(seconds))

    if scoreLasso >= topScoreLasso:
        topScoreLasso = scoreLasso
        bestCLasso = c
    
    if scoreRidge >= topScoreRidge:
        topScoreRidge = scoreRidge
        bestCRidge = c

    print(index, " - ", maxIndex, ". C = ", c)
    print("Score Lasso: ", scoreLasso, " | Time Lasso: ", timeTakenLassoDisplay)
    print("Score Ridge: ", scoreRidge, " | Time Ridge: ", timeTakenRidgeDisplay)
    print("Time Taken Index: ", timeTakenDisplay)
    print("-----------------------------------------------------")
    
    index = index + 1

    
dlMuitoBoas_LancasterStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = bestCLasso, tol = 1e-3)
dlMuitoBoas_LancasterStem_Lasso.fit(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas)

dlMuitoBoas_LancasterStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = bestCRidge, tol = 1e-3)
dlMuitoBoas_LancasterStem_Ridge.fit(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print()
print("---- Classificação Binária - Cervejas Muito Boas ----")
print("Regularização Lasso")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_LancasterStem_Lasso.score(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_LancasterStem_Lasso.score(X1b_MuitoBoas_lancaster, y1bBinClass_MuitoBoas) * 100)
print("Melhor C: ", bestCLasso)
print("Tempo Demorado: ", timeTakenLassoDisplay)
print("--------------------------")
print("Regularização Ridge")
print("% Acertos nos Dados de Treino: ", dlMuitoBoas_LancasterStem_Ridge.score(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoBoas_LancasterStem_Ridge.score(X1b_MuitoBoas_lancaster, y1bBinClass_MuitoBoas) * 100)
print("Melhor C: ", bestCRidge)
print("Tempo Demorado: ", timeTakenRidgeDisplay)
print("--------------------------")
print("Total Tempo Demorado: ", timeTakenTotalDisplay)

# print()
# print("---- Classificação Binária - Cervejas Muito Más ----")
# print("% Acertos nos Dados de Treino: ", dlMuitoMas.score(X_Train_tfidf, y_Train_ClassBin_MuitoMas) * 100)
# print("% Acertos nos Dados de Teste: ", dlMuitoMas.score(X_Test_tfidf, y_Test_ClassBin_MuitoMas) * 100)

1  -  9 . C =  0.1
Score Lasso:  0.94972  | Time Lasso:  0:1
Score Ridge:  0.94968  | Time Ridge:  0:0
Time Taken Index:  0:2
-----------------------------------------------------
2  -  9 . C =  0.5
Score Lasso:  0.94928  | Time Lasso:  0:3
Score Ridge:  0.94956  | Time Ridge:  0:0
Time Taken Index:  0:4
-----------------------------------------------------
3  -  9 . C =  1
Score Lasso:  0.949  | Time Lasso:  0:14
Score Ridge:  0.94936  | Time Ridge:  0:1
Time Taken Index:  0:15
-----------------------------------------------------
4  -  9 . C =  2.5
Score Lasso:  0.94776  | Time Lasso:  0:37
Score Ridge:  0.94824  | Time Ridge:  0:1
Time Taken Index:  0:39
-----------------------------------------------------
5  -  9 . C =  5
Score Lasso:  0.94404  | Time Lasso:  1:54
Score Ridge:  0.94776  | Time Ridge:  0:45
Time Taken Index:  2:39
-----------------------------------------------------
6  -  9 . C =  10
Score Lasso:  0.93744  | Time Lasso:  4:33
Score Ridge:  0.94628  | Time Ridge:  

Uma conclusão rápida a ser tirada é a diferença abismal de tempo demorado a treinar o classificador quando usada a regularização Lasso. Isto porque o Lasso tem uma adaptação de step.<br>
Assim, e tendo em conta que as percentagens de acerto são semelhantes, vamos optar pela regularização Ridge.


<b>Não conseguimos tirar grandes conclusões daqui.<br>
Explicar que as criticas muito boas são uma infima percentagem e temos de observar a matriz de confusão.
</b>

In [155]:
dl_MuitoBoas_NoStem_List = [None] * len(ListC)
dl_MuitoBoas_PorterStemmer_List = [None] * len(ListC)
dl_MuitoBoas_SnowballStemmer_List = [None] * len(ListC)
dl_MuitoBoas_LancasterStemmer_List =[None] * len(ListC)

ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100]
index = 1
begin = time.time()

for c in ListC:
    timeBeginIndex = time.time()

    #No Stem
    dl_MuitoBoas_NoStem = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dl_MuitoBoas_NoStem.fit(X1a_MuitoBoas_noStem, y1aBinClass_MuitoBoas)
    
    #Porter Stemmer
    dl_MuitoBoas_PorterStemmer = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dl_MuitoBoas_PorterStemmer.fit(X1a_MuitoBoas_porter, y1aBinClass_MuitoBoas)
    
    #Snowball Stemmer
    dl_MuitoBoas_SnowballStemmer = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dl_MuitoBoas_SnowballStemmer.fit(X1a_MuitoBoas_snowball, y1aBinClass_MuitoBoas)
    
    #Lancaster Stemmer
    dl_MuitoBoas_LancasterStemmer = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dl_MuitoBoas_LancasterStemmer.fit(X1a_MuitoBoas_lancaster, y1aBinClass_MuitoBoas)
    
    timeEndIndex = time.time()
    
    #Time
    timeTakenIndex = timeEndIndex - timeBeginIndex
    minutes = np.floor(timeTakenIndex / 60)
    seconds = timeTakenIndex % 60
    timeTakenIndexDisplay = str(int(minutes)) + ":" + str(int(seconds))
    
    print(index, " - ", maxIndex, ". C = ", c)
    print("Time Taken Index: ", timeTakenIndexDisplay)
    print("-----------------------------------------------------")
    
    #Indexar Modelos
    dl_MuitoBoas_NoStem_List[index - 1] = dl_MuitoBoas_NoStem
    dl_MuitoBoas_PorterStemmer_List[index - 1] = dl_MuitoBoas_PorterStemmer
    dl_MuitoBoas_SnowballStemmer_List[index - 1] = dl_MuitoBoas_SnowballStemmer
    dl_MuitoBoas_LancasterStemmer_List[index - 1] = dl_MuitoBoas_LancasterStemmer
    
    index = index + 1

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenTotalDisplay = str(int(minutes)) + ":" + str(int(seconds))

print()
print("Total Tempo Demorado: ", timeTakenTotalDisplay)

1  -  9 . C =  0.1
Time Taken Index:  0:3
-----------------------------------------------------
2  -  9 . C =  0.5
Time Taken Index:  0:3
-----------------------------------------------------
3  -  9 . C =  1
Time Taken Index:  0:5
-----------------------------------------------------
4  -  9 . C =  2.5
Time Taken Index:  0:7
-----------------------------------------------------
5  -  9 . C =  5
Time Taken Index:  0:11
-----------------------------------------------------
6  -  9 . C =  10
Time Taken Index:  0:15
-----------------------------------------------------
7  -  9 . C =  25
Time Taken Index:  0:22
-----------------------------------------------------
8  -  9 . C =  50
Time Taken Index:  0:27
-----------------------------------------------------
9  -  9 . C =  100
Time Taken Index:  0:34
-----------------------------------------------------

Total Tempo Demorado:  2:12


#### Classificação Binária: Criticas Muito Mas

##### No Stem

In [39]:
#Forçar hiperparâmetros - Muito Boas
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScoreLasso = 0
topScoreRidge = 0
index = 1
maxIndex = len(ListC)

begin = time.time()

for c in ListC:
    timeBeginClass = time.time()

    #Lasso
    beginLasso = time.time()
    dlMuitoMas_NoStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoMas_NoStem_Lasso.fit(X1a_MuitoMas_noStem, y1aBinClass_MuitoMas)
    scoreLasso = dlMuitoMas_NoStem_Lasso.score(X1b_MuitoMas_noStem, y1bBinClass_MuitoMas)
    endLasso = time.time()

    #Time - Lasso
    timeTakenLasso = endLasso - beginLasso
    minutesLasso = np.floor(timeTakenLasso / 60)
    secondsLasso = timeTakenLasso % 60
    timeTakenLassoDisplay = str(int(minutesLasso)) + ":" + str(int(secondsLasso))
   
    #Ridge
    beginRidge = time.time()
    dlMuitoMas_NoStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = c, tol = 1e-3)
    dlMuitoMas_NoStem_Ridge.fit(X1a_MuitoMas_noStem, y1aBinClass_MuitoMas)
    scoreRidge = dlMuitoMas_NoStem_Ridge.score(X1b_MuitoMas_noStem, y1bBinClass_MuitoMas)
    endRidge = time.time()
    
    #Time - Ridge
    timeTakenRidge = endRidge - beginRidge
    minutesRidge = np.floor(timeTakenRidge / 60)
    secondsRidge = timeTakenRidge % 60
    timeTakenRidgeDisplay = str(int(minutesRidge)) + ":" + str(int(secondsRidge))

    #Time - Total
    timeTakenClass = time.time() - timeBeginClass
    minutes = np.floor(timeTakenClass / 60)
    seconds = timeTakenClass % 60
    timeTakenDisplay = str(int(minutes)) + ":" + str(int(seconds))

    if scoreLasso >= topScoreLasso:
        topScoreLasso = scoreLasso
        bestCLasso = c
    
    if scoreRidge >= topScoreRidge:
        topScoreRidge = scoreRidge
        bestCRidge = c

    print(index, " - ", maxIndex, ". C = ", c)
    print("Score Lasso: ", scoreLasso, " | Time Lasso: ", timeTakenLassoDisplay)
    print("Score Ridge: ", scoreRidge, " | Time Ridge: ", timeTakenRidgeDisplay)
    print("Time Taken Index: ", timeTakenDisplay)
    print("-----------------------------------------------------")
    
    index = index + 1

    
dlMuitoMas_NoStem_Lasso = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 1000, C = bestCLasso, tol = 1e-3)
dlMuitoMas_NoStem_Lasso.fit(X1a_MuitoMas_noStem, y1aBinClass_MuitoMas)

dlMuitoMas_NoStem_Ridge = LogisticRegression(penalty = 'l2', max_iter = 1000, C = bestCRidge, tol = 1e-3)
dlMuitoMas_NoStem_Ridge.fit(X1a_MuitoMas_noStem, y1aBinClass_MuitoMas)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(int(seconds))

print()
print("---- Classificação Binária - Cervejas Muito Mas ----")
print("Regularização Lasso")
print("% Acertos nos Dados de Treino: ", dlMuitoMas_NoStem_Lasso.score(X1a_MuitoMas_noStem, y1aBinClass_MuitoMas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoMas_NoStem_Lasso.score(X1b_MuitoMas_noStem, y1bBinClass_MuitoMas) * 100)
print("Melhor C: ", bestCLasso)
print("Tempo Demorado: ", timeTakenLassoDisplay)
print("--------------------------")
print("Regularização Ridge")
print("% Acertos nos Dados de Treino: ", dlMuitoMas_NoStem_Ridge.score(X1a_MuitoMas_noStem, y1aBinClass_MuitoMas) * 100)
print("% Acertos nos Dados de Validação: ", dlMuitoMas_NoStem_Ridge.score(X1b_MuitoMas_noStem, y1bBinClass_MuitoMas) * 100)
print("Melhor C: ", bestCRidge)
print("Tempo Demorado: ", timeTakenRidgeDisplay)
print("--------------------------")
print("Total Tempo Demorado: ", timeTakenTotalDisplay)

# print()
# print("---- Classificação Binária - Cervejas Muito Más ----")
# print("% Acertos nos Dados de Treino: ", dlMuitoMas.score(X_Train_tfidf, y_Train_ClassBin_MuitoMas) * 100)
# print("% Acertos nos Dados de Teste: ", dlMuitoMas.score(X_Test_tfidf, y_Test_ClassBin_MuitoMas) * 100)

1  -  10 . C =  0.1
Score Lasso:  0.93848  | Time Lasso:  0:1
Score Ridge:  0.93836  | Time Ridge:  0:1
Time Taken Index:  0:2
-----------------------------------------------------
2  -  10 . C =  0.5
Score Lasso:  0.94112  | Time Lasso:  0:4
Score Ridge:  0.94136  | Time Ridge:  0:0
Time Taken Index:  0:5
-----------------------------------------------------
3  -  10 . C =  1
Score Lasso:  0.94328  | Time Lasso:  0:10
Score Ridge:  0.9428  | Time Ridge:  0:1
Time Taken Index:  0:11
-----------------------------------------------------
4  -  10 . C =  2.5
Score Lasso:  0.94296  | Time Lasso:  1:8
Score Ridge:  0.94408  | Time Ridge:  0:2
Time Taken Index:  1:10
-----------------------------------------------------
5  -  10 . C =  5
Score Lasso:  0.93952  | Time Lasso:  3:23
Score Ridge:  0.94368  | Time Ridge:  0:2
Time Taken Index:  3:26
-----------------------------------------------------
6  -  10 . C =  10
Score Lasso:  0.93184  | Time Lasso:  7:20
Score Ridge:  0.94172  | Time Rid

In [ ]:
#Forçar hiperparâmetros
dl_porter = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 5000, C = 10)
dl_porter.fit(X1a_porter, ya_Train_ClassBin)

print(dl_porter.score(X1a_porter, ya_Train_ClassBin))
print(dl_porter.score(X1b_porter, yb_Train_ClassBin))
print()


dl_snowball = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 5000, C = 10)
dl_snowball.fit(X1a_snowball, ya_Train_ClassBin)

print(dl_snowball.score(X1a_snowball, ya_Train_ClassBin))
print(dl_snowball.score(X1b_snowball, yb_Train_ClassBin))
print()


dl_lancaster = LogisticRegression(penalty = 'l1', solver='saga', max_iter = 5000, C = 10)
dl_lancaster.fit(X1a_lancaster, ya_Train_ClassBin)

print(dl_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print(dl_lancaster.score(X1b_lancaster, yb_Train_ClassBin))

### SVM

In [ ]:
# snowballStemFunc = SnowballStemmer(language='english')
# Docs1_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs1]
# Docs2_SnowballStem =[' '.join([snowballStemFunc.stem(w) for w in d.split()]) for d in Docs2]

# tfidf_snowball = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w+\b', ngram_range=(1, 3)).fit(Docs1_SnowballStem)
# X1_snowball = tfidf_snowball.transform(Docs1_SnowballStem)
# X2_snowball = tfidf_snowball.transform(Docs2_SnowballStem)
# tokens_snowball = tfidf_snowball.get_feature_names()

#begin = time.time()

svm_snowball = SVC(kernel = 'rbf', gamma = 0.25, C = 1).fit(X1_snowball, y_Train_ClassBin)
         
#end = time.time()

# timeTaken = end - begin
# minutes = np.floor(timeTaken / 60)
# seconds = timeTaken % 60
# timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

#print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1_snowball, y_Train_ClassBin))
print("% de acertos no teste: ", svm_snowball.score(X2_snowball, y_Test_ClassBin))
#print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

### No Stem

In [ ]:
tfidf_noStem = TfidfVectorizer(min_df = 3, token_pattern = r'\b\w\w\w+\b').fit(Docs1a)
X1a_noStem = tfidf_noStem.transform(Docs1a)
X1b_noStem = tfidf_noStem.transform(Docs1b)
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_noStem = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_noStem, ya_Train_ClassBin)
        score = svm_noStem.score(X1b_noStem, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_noStem = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_noStem, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_noStem.score(X1a_noStem, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

### Porter Stemmer

In [ ]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 0
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_porter = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_porter, ya_Train_ClassBin)
        score = svm_porter.score(X1b_porter, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_porter = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_porter, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_porter.score(X1a_porter, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

### Snowball Stemmer

In [ ]:
begin = time.time()

topScore = 0
ListG = [0.05, 0.1, 0.25, 0.5, 0.75, 1, 'auto']
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_snowball = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_snowball, ya_Train_ClassBin)
        score = svm_snowball.score(X1b_snowball, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_snowball = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_snowball, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_snowball.score(X1a_snowball, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

### Lancaster Stemmer

In [ ]:
begin = time.time()

topScore = 0
ListG = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
ListC = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250]
topScore = 0
index = 1
maxIndex = len(ListG) * len(ListC)

for c in ListC:
    for g in ListG:
        timeBeginClass = time.time()
        
        svm_lancaster = SVC(kernel = 'rbf', gamma = g, C = c).fit(X1a_lancaster, ya_Train_ClassBin)
        score = svm_lancaster.score(X1b_lancaster, yb_Train_ClassBin)
        
        timeTakenClass = time.time() - timeBeginClass
        minutes = np.floor(timeTakenClass / 60)
        seconds = timeTakenClass % 60
        timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)
        
        if score >= topScore:
            topScore = score
            par = (g, c)
            
        print(index, " - ", maxIndex, ". G = ", g, " | C = ", c, " | Score:", score, " | Time: ", timeTakenDisplay)
        index = index + 1
            
svm_lancaster = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1a_lancaster, ya_Train_ClassBin)

end = time.time()

timeTaken = end - begin
minutes = np.floor(timeTaken / 60)
seconds = timeTaken % 60
timeTakenDisplay = str(int(minutes)) + ":" + str(seconds)

print("Tempo Demorado: ", timeTakenDisplay)
print("% de acertos no treino: ", svm_lancaster.score(X1a_lancaster, ya_Train_ClassBin))
print("% de acertos na validação: ", topScore)
print("Parametros: gamma=", par[0], ", C=", par[1])

# svm = SVC(kernel = 'rbf', gamma = par[0], C = par[1]).fit(X1n, y1)
# print("% de acertos no treino: ", svm.score(X1n, y1))
# print("% de acertos no teste: ", svm.score(X2n, y2))
# print("Parametros: gamma=%.4f, C=%.1f: "%(par[0], par[1]))

# Notas
## Labs
finalLab = (4.12 + 9.47 + 14.12 + 15 + 20) / 5 

## Casa
finalCasa = (15.5 + 7.5 + 11.5 + 3.5) / 74 * 20

print(finalLab)
print(finalCasa)

print("Media Final, sem trabalho: ", (finalLab + finalCasa) / 2)
print("Media Final, com percentagens: ", (finalLab + finalCasa) / 2 / 2)

# Demonstração do Engenheiro Na Aula - Projecto

In [66]:
import numpy as np
import pickle

In [68]:
D_train = pickle.load(open('rateBeer75Ktrain.p', 'rb'))
D_test = pickle.load(open('rateBeer25Ktest.p', 'rb'))

In [71]:
k = list(D_train.keys())

print(k[:10])

['000000', '000001', '000002', '000003', '000004', '000005', '000006', '000007', '000008', '000009']


In [73]:
D_train[k[0]]

{'name': 'Anderson Valley Brother Davids Double',
 'style': 'Abbey Dubbel',
 'who': 'dmtroyer',
 'feel': '3/5',
 'look': '3/5',
 'smell': '2/5',
 'taste': '3/5',
 'overall': '4/10',
 'review': '22oz.  Pours dark brown with a red rim and off-white head.  Nose is mostly sweet malt with fairly noticeable alcohol depending on the temperature.  Flavor is sweet malt with raisin and banana yeast with some slight spice on the finish.  Very true to the style, but less than inspiring.'}

In [76]:
y = np.array([D_train[k]['overall'][:-3] for k in D_train], dtype='uint8')

print(y[:10])

[4 6 4 8 7 8 3 5 5 3]


In [77]:
Docs = [D_train[k]['review'] for k in D_train]

print(len(Docs))

75000


In [78]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
tfidf = TfidfVectorizer().fit(Docs)

In [80]:
voc = tfidf.get_feature_names()

len(voc)

58138

In [81]:
print(voc[:100])

['00', '000', '00000000000', '00007738', '0001', '000th', '001', '0013', '001880', '0019', '002', '003', '00802a1ba', '00am', '01', '010308', '010509', '010607', '010910', '011', '013', '014', '01501', '01546', '016', '01633', '017', '019', '01935', '01996', '01se11', '02', '020', '020202', '020310', '02032010', '02051', '020606', '020803', '020907', '021010', '021107', '021110', '022310', '0233', '02482208', '02feb07', '02pouring', '02s', '03', '030110', '030208', '030303', '030407', '030707', '030905', '030911', '031108', '032460', '033', '03a', '04', '040205', '040311', '040402', '040404', '040607', '040611', '040808', '0489', '04feb08', '04holy', '04may10', '04s', '05', '050', '050111', '050135', '050211', '050411', '05042010', '050505', '050505s', '050608', '050808', '050908', '051001', '051028', '051207', '058', '05905', '05brilliantly', '05ma10', '05mar10', '05s', '06', '060113', '060407', '060510', '060518']


In [82]:
print(voc[10000:10100])

['buiscut', 'buiscuty', 'buit', 'bukowskis', 'bulb', 'bulbous', 'bulbs', 'bulging', 'bulk', 'bulked', 'bulkier', 'bulking', 'bulkington', 'bulky', 'bull', 'bulldog', 'bulldogp04', 'bulldogs', 'bulle', 'bulles', 'bullet', 'bullfrog', 'bullfrogs', 'bullied', 'bullion', 'bullit', 'bullocks', 'bulls', 'bullseye', 'bullshit', 'bully', 'bullying', 'bulwark', 'bum', 'bumble', 'bumcrack', 'bummed', 'bummer', 'bump', 'bumped', 'bumping', 'bumps', 'bumpy', 'bums', 'bumwines', 'bun', 'bunch', 'bunched', 'bunches', 'bundet', 'bundle', 'bundled', 'bundt', 'bunhc', 'bunk', 'bunker', 'bunkypop', 'bunn', 'bunny', 'bunr', 'buns', 'bunt', 'bunting', 'bunyanesque', 'buon', 'buona', 'buone', 'buono', 'buoy', 'buoyancy', 'buoyant', 'buoyed', 'bur', 'burb', 'burbage', 'burbank', 'burbled', 'burbles', 'burbling', 'burbon', 'burbonic', 'burboun', 'burbs', 'burden', 'burdened', 'burdening', 'burdensome', 'burdensomely', 'burdgundy', 'burdoch', 'burdock', 'burdonsome', 'burek', 'buren', 'burgandy', 'burgatory',

In [83]:
tfidf = TfidfVectorizer(min_df=5).fit(Docs)

In [84]:
voc = tfidf.get_feature_names()

len(voc)

15858

In [87]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 2)).fit(Docs)

In [88]:
voc = tfidf.get_feature_names()

len(voc)

129925

In [90]:
print(voc[10000:10100])

['are incredibly', 'are indeed', 'are initially', 'are intense', 'are intensely', 'are interesting', 'are into', 'are it', 'are its', 'are joined', 'are just', 'are kept', 'are kind', 'are kinda', 'are known', 'are lacking', 'are largely', 'are layered', 'are left', 'are less', 'are light', 'are lightly', 'are like', 'are little', 'are long', 'are looking', 'are lot', 'are lots', 'are lovely', 'are low', 'are lucky', 'are made', 'are mainly', 'are making', 'are malt', 'are malty', 'are many', 'are medium', 'are mellow', 'are met', 'are mild', 'are mildly', 'are minimal', 'are missing', 'are mix', 'are mixed', 'are moderate', 'are more', 'are most', 'are mostly', 'are much', 'are muted', 'are my', 'are nearly', 'are never', 'are new', 'are nice', 'are nicely', 'are no', 'are not', 'are notable', 'are noted', 'are notes', 'are nothing', 'are noticable', 'are noticeable', 'are noticed', 'are now', 'are nowhere', 'are nutty', 'are obvious', 'are of', 'are off', 'are often', 'are ok', 'are 

In [91]:
print(voc[50000:50100])

['grain burnt', 'grain but', 'grain caramel', 'grain cereal', 'grain character', 'grain chocolate', 'grain citrus', 'grain cocoa', 'grain coffee', 'grain cookie', 'grain corn', 'grain dark', 'grain dry', 'grain finish', 'grain flavor', 'grain flavors', 'grain floral', 'grain for', 'grain fruit', 'grain fruity', 'grain grass', 'grain grassy', 'grain hay', 'grain heavy', 'grain here', 'grain hint', 'grain hints', 'grain honey', 'grain hop', 'grain hoppy', 'grain hops', 'grain husk', 'grain husks', 'grain in', 'grain is', 'grain it', 'grain its', 'grain lemon', 'grain light', 'grain lightly', 'grain like', 'grain little', 'grain malt', 'grain malts', 'grain meal', 'grain medium', 'grain mild', 'grain mixed', 'grain moderately', 'grain molasses', 'grain more', 'grain mouthfeel', 'grain nice', 'grain nose', 'grain not', 'grain note', 'grain notes', 'grain nuts', 'grain of', 'grain on', 'grain or', 'grain orange', 'grain palate', 'grain pale', 'grain presence', 'grain pretty', 'grain profile

In [92]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 2), max_features=20000).fit(Docs)

In [93]:
voc = tfidf.get_feature_names()

print(len(voc))

20000


In [94]:
print(voc[10000:10100])

['left behind', 'left coast', 'left good', 'left in', 'left light', 'left me', 'left nice', 'left on', 'left some', 'left the', 'left with', 'legend', 'legs', 'lemon', 'lemon and', 'lemon citrus', 'lemon grapefruit', 'lemon grass', 'lemon in', 'lemon juice', 'lemon lime', 'lemon orange', 'lemon peel', 'lemon rind', 'lemon some', 'lemon the', 'lemon with', 'lemon zest', 'lemonade', 'lemongrass', 'lemons', 'lemons and', 'lemony', 'lemony and', 'lemony citrus', 'lend', 'lending', 'lends', 'length', 'lengthy', 'lengthy aftertaste', 'lengthy finish', 'les', 'less', 'less of', 'less sweet', 'less than', 'lesser', 'let', 'let down', 'let it', 'let me', 'let the', 'let this', 'let you', 'letdown', 'lets', 'lets see', 'letting', 'level', 'level is', 'level of', 'levels', 'levels of', 'levure', 'lger', 'lger de', 'lger got', 'lgre', 'lgrement', 'liars', 'liars club', 'licorice', 'licorice and', 'licorice flavor', 'licorice in', 'licorice notes', 'licorice taste', 'licorice the', 'licorice with',

In [95]:
D1, D2, y1, y2 = train_test_split(Docs, y, test_size=1/3, stratify=y)

In [96]:
print(len(D1))
print(len(D2))

50000
25000


In [97]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 2), max_features=20000).fit(D1)
X1 = tfidf.transform(D1)
X2 = tfidf.transform(D2)

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [101]:
y1b = (y1 <= 2) * 1
y2b = (y2 <= 2) * 1

In [102]:
lr = LogisticRegression().fit(X1, y1b)

In [103]:
y2e = lr.predict(X2)

In [107]:
CM = confusion_matrix(y2b, y2e)

print(CM)

[[23859    26]
 [  995   120]]


Esta matriz está ao contrário.

A matriz está:

TN | FP -> Classe 0<br>
FN | TP -> Classe 1

In [108]:
print(np.sum(y1b == 1))

2231


In [109]:
print(np.sum(y2b==y2e) / y2b.shape[0] * 100)

95.916


In [110]:
np.sum(y2b==1) #Existem 1115 negativos

1115

In [111]:
1115 / 25000 #Isto significa que só existem 5% de criticas 

0.0446

In [112]:
w = lr.coef_
w = w.squeeze()

print(w.shape)

(20000,)


In [113]:
idx = np.argsort(w)

In [114]:
print(w[idx[:10]])

[-5.40987932 -4.1001349  -3.36851137 -3.29324408 -3.17878706 -3.11660772
 -3.06789631 -2.89637988 -2.60612801 -2.53713289]


In [115]:
print(w[idx[-10:]])

[3.30011339 3.3270514  3.33677448 3.37560143 3.41120253 3.65119641
 3.72895905 3.7750522  5.07887283 5.45684473]


In [117]:
vg = [voc[i] for i in idx[:30]] # Boas
vb = [voc[i] for i in idx[-30:]] # Mas

In [119]:
print(vg)

['nice brew', 'citrus grass', 'chocolate character', 'smooth feel', 'not expecting', 'great sweet', 'hops good', 'dark cloudy', 'solid porter', 'complexity of', 'hops are', 'spicy floral', 'roasted sweet', 'tea', 'coffee flavors', 'well balanced', 'ipa is', 'grapefruity', 'hop spiciness', 'caramel bit', 'no apparent', 'good flavor', 'pine needle', 'fridge', 'rich golden', 'that', 'releases', 'expect in', 'apple cider', 'fully']


In [120]:
print(vb)

['very nutty', 'interesting', 'both the', 'yeasty spicy', 'distance', 'rather than', 'porter that', 'slight bourbon', 'awful', 'all it', 'drink it', 'thin collar', 'bacon', 'bad beer', 'slight chocolate', 'interested', 'not dry', 'watermelon', 'stands out', 'drink in', 'blend', 'up as', 'carried', 'no malt', 'last years', 'midway', 'necessarily', 'not my', 'watery carbonation', 'could not']


In [121]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 2), max_features=20000).fit(Docs)

In [123]:
X1 = tfidf.transform(D1)
X2 = tfidf.transform(D2)

In [124]:
lr = LogisticRegression().fit(X1, y1b)

w = lr.coef_.squeeze()

idx = np.argsort(w)

voc = tfidf.get_feature_names()

In [125]:
vg = [voc[i] for i in idx[:30]] # Boas
vb = [voc[i] for i in idx[-30:]] # Mas

In [126]:
print(vb)

['very little', 'infection', 'boring', 'yellow', 'disappointing', 'raspberry', 'poor', 'skunk', 'alky', 'awful', 'thin', 'drain pour', 'bad', 'bad bottle', 'skunky', 'infected', 'not', 'water', 'stale', 'drain', 'bland', 'unpleasant', 'cardboard', 'no', 'lager', 'metallic', 'nasty', 'not good', 'watery', 'corn']


In [128]:
y2e = lr.predict(X2)

CM = confusion_matrix(y2b, y2e)

print(CM)

[[23858    27]
 [  998   117]]


In [129]:
from sklearn.linear_model import LinearRegression

In [131]:
lr = LinearRegression().fit(X1, y1)

In [132]:
y2e = lr.predict(X2)

In [133]:
print(lr.score(X2, y2))

0.25599983590407993


score modelos classificacoes: percentagem acertos

score modelos regressao: R2 (coeficiente de determinação) = Potência do erro / Variancia do Y.<br>
Se R2 for negativo, indica que as coisas estão a correr muito mal.

In [135]:
print(y2e.min())
print(y2e.max())

-1.8412977611875831
12.694176213756613


In [136]:
from sklearn.metrics import roc_curve

In [137]:
lr = LogisticRegression().fit(X1, y1b)

In [138]:
y2e = lr.predict(X2)

In [139]:
print(confusion_matrix(y2b, y2e))

[[23858    27]
 [  998   117]]


In [140]:
y2d = lr.decision_function(X2)

print(confusion_matrix(y2b, y2d > -1))

[[23654   231]
 [  757   358]]
